<a href="https://colab.research.google.com/github/zeoxleh/RAGAS-as-a-Framework-for-Assessing-Retrieval-Augmented-Generation-in-Materials-Science-Applications/blob/main/Gemini_RAG_RAGAS_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install & import

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install -U -q google-generativeai
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
# Install the client library and import necessary modules.
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Set the API key

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Test run to check Gemini

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro')

In [ ]:
%%time
response = model.generate_content("What is graphene?")

CPU times: user 127 ms, sys: 14.9 ms, total: 142 ms
Wall time: 8.15 s


In [ ]:
to_markdown(response.text)

> Graphene is a single layer of carbon atoms arranged in a two-dimensional honeycomb lattice.  Think of it like chicken wire, but made of carbon and at the atomic scale.  This simple structure gives it a remarkable combination of properties:
> 
> * **Exceptional Strength:**  It's about 200 times stronger than steel by weight.
> * **Lightweight:**  A single sheet is practically transparent and weighs almost nothing.
> * **Excellent Electrical Conductivity:** Electrons move through graphene incredibly quickly, making it a superb conductor.
> * **High Thermal Conductivity:** It dissipates heat very effectively.
> * **Flexible and Transparent:**  It can be bent and stretched while remaining transparent.
> * **Impermeable:**  Even helium atoms can't pass through a perfect sheet of graphene.
> 
> These properties make graphene a promising material for a wide range of applications, including:
> 
> * **Electronics:**  Faster transistors, flexible displays, sensors.
> * **Energy Storage:**  Improved batteries and supercapacitors.
> * **Composite Materials:**  Stronger and lighter plastics, coatings.
> * **Biomedical Applications:**  Drug delivery, tissue engineering.
> * **Water Filtration:**  Membranes for desalination and purification.
> 
> While graphene holds enormous potential, there are still challenges in producing it at a large scale and integrating it into practical devices.  Research is ongoing to overcome these hurdles and unlock the full potential of this remarkable material.


###Embedding

In [ ]:
from typing import List
import numpy as np

def embed_docs(docs: List[str]) -> List[List[float]]:
    res = genai.embed_content(
    model="models/text-embedding-004",
    content=docs,
    task_type="retrieval_document")
    embeddings = res['embedding']
    return embeddings

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os

pinecone = Pinecone(api_key="API_KEY")

In [ ]:
import time

index_name = "context-index"
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1") )

# wait for index to finish initialization
while not pinecone.describe_index(index_name).status["ready"]:
    time.sleep(1)


In [ ]:
import pandas as pd

df_knowledge = pd.read_csv("Combine_Parsed.csv")
df_knowledge.head()

,Final
0,Author:\nMustafa K. Bayazit\n\nDoi:\n10.1021/a...
1,Author:\nPanpan Zhang\n\nDoi:\n10.1021/acsnano...
2,Author:\nYuan Huang\n\nDoi:\n10.1021/acsnano.5...
3,Author:\nSatyaprakash Ahirwar\n\nDoi:\n10.1021...
4,Author:\nSheng Yang\n\nDoi:\n10.1021/jacs.5b09...


In [ ]:
from tqdm.auto import tqdm

batch_size = 1
vector_limit = 1000

answers = df_knowledge[:vector_limit]
index = pinecone.Index(index_name)

for i in tqdm(range(0, len(answers), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(answers))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{"text": text} for text in answers["Final"][i:i_end]]
    # create embeddings
    texts = answers["Final"][i:i_end].tolist()
    embeddings = embed_docs(texts)
    # create records list for upsert
    records = zip(ids, embeddings, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# check number of records in the index
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}

In [ ]:
result = index.fetch(ids=["99"])
print(result)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'99': {'id': '99',
                    'metadata': {'text': 'Author:\n'
                                         'Ding Zhou\n'
                                         '\n'
                                         'DOI:\n'
                                         'https://doi-org.remotexs.ntu.edu.sg/10.1038/ncomms5716\n'
                                         '\n'
                                         'Method of producing graphene:\n'
                                         'Porous graphene (pGRF) was prepared '
                                         'by using oxometalates (OM where '
                                         'M=Mo, W, Al, Mn, Sn, Ti or V) as the '
                                         'etching agents. Aqueous graphene '
                                         'oxide (GO) dispersion was prepared '
                                         'by the modified Hummers’ method. OM '
                       

###Creating response

In [ ]:
def print_messages(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}\n\n{bold}> Output{unbold}\n{response}\n")

In [ ]:
separator = "\n"

def construct_context(contexts: List[str]) -> str:
    chosen_sections = []
    chosen_sections_len = 0

    for text in contexts:
        text = text.strip()
        chosen_sections_len += len(text) + 2
        chosen_sections.append(text + '\n\n')
    concatenated_doc = separator.join(chosen_sections)

    return concatenated_doc

In [ ]:
def create_payload(question, context_str):
    prompt_template = """
INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each contextare formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
{context}


QUESTION:
{question}"""

    text_input = prompt_template.replace("{context}", context_str).replace("{question}", question)

    return text_input

In [ ]:
def rag_query(question: str) -> str:
    # create query vec
    query_vec = embed_docs(question)
    # query pinecone
    res = index.query(vector= query_vec, top_k=3, include_metadata=True)
    # get contexts
    contexts = [match.metadata["text"] for match in res.matches]
    # build the multiple contexts string
    context_str = construct_context(contexts=contexts)
    # create our retrieval augmented prompt
    payload = create_payload(question,context_str)
    response = model.generate_content(payload)
    print_messages(payload, response.text)

### Initialise RAGAS


In [ ]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1

In [ ]:
pip install langchain-openai

In [ ]:
from datasets import Dataset

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY"

In [ ]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))


In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas import evaluate
metrics = [
    LLMContextRecall(llm=evaluator_llm),
    FactualCorrectness(llm=evaluator_llm),
    Faithfulness(llm=evaluator_llm),
]

## Test set from ragas

### Question 1


In [ ]:
rag_query("What is the purpose of the quartz reactor in the method of producing graphene?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Teresa Campo

Doi: 
https://doi.org/10.3844/ajeassp.2016.574.583

Method of producing graphene:
After cleaning, copper substrates were placed into a quartz boat and introduced in a cylindrical quartz reactor (inner diameter of 25 mm and 1 m-length) within a tube furnace. The quartz reactor was maintained at an initial vacuum of 10^-2 Torr using a mechanical pump. The temperature was risen to 950°C, at a rate of 30°C/min (Guermoune et al., 2011). Samples were maintained at this temperature for 26 min, using an H2 constant flow of 10 sccm to remove possible organic impurities. After the first 20 min of this step, 

In [ ]:
from datasets import Dataset

data_samples = {
    'question': ['What is the purpose of the quartz reactor in the method of producing graphene?'],
    'answer': ['''The most relevant context to answer this question is the one by **Teresa Campo**.

**Purpose of the Quartz Reactor:**

The quartz reactor serves several key purposes in the graphene production process described:

1. **Containment Vessel:** It acts as a controlled environment chamber, separating the reaction from the external atmosphere. This isolation is crucial for maintaining the desired gas composition and pressure throughout the process.

2. **High-Temperature Resistance:** Quartz can withstand the high temperatures (950°C in this case) required for the growth of graphene on the copper substrate.

3. **Chemical Inertness:** Quartz is relatively chemically inert, meaning it won't react with the gases or the copper substrate at high temperatures, ensuring the purity of the produced graphene.

4. **Plasma Generation:**  The context mentions a "blue plasma discharge" generated within the reactor. This implies that the quartz reactor might be designed to facilitate plasma-enhanced chemical vapor deposition (PECVD), where the plasma aids in the decomposition of precursor gases (ethanol in this case) and enhances the graphene growth process.

**In summary,** the quartz reactor provides a controlled, high-temperature, chemically inert environment that is potentially suitable for plasma enhancement, all crucial factors for the successful growth of graphene.

'''],

    'contexts' : [['''Author:
Teresa Campo

Doi:
https://doi.org/10.3844/ajeassp.2016.574.583

Method of producing graphene:
After cleaning, copper substrates were placed into a quartz boat and introduced in a cylindrical quartz reactor (inner diameter of 25 mm and 1 m-length) within a tube furnace. The quartz reactor was maintained at an initial vacuum of 10^-2 Torr using a mechanical pump. The temperature was risen to 950°C, at a rate of 30°C/min (Guermoune et al., 2011). Samples were maintained at this temperature for 26 min, using an H2 constant flow of 10 sccm to remove possible organic impurities. After the first 20 min of this step, a flow mixture of N2 gas and ethanol vapor (30 sccm), previously generated by an overpressure of N2, was introduced into the reactor. In this moment, a blue plasma discharge was generated. At the same time, the reactor pressure increases to 102 Torr during a few seconds until the pressure stabilizes at 1 Torr. Once the growth time finishes, a cooling process takes place. During the first 30 min of the cooling step, a constant flow of H2 10 sccm is maintained and, after that, an Ar flow of 300 sccm is introduced into the reactor.''',


'''Author:
Tommi Kaplas

Doi:
https://doi.org/10.1016/j.carbon.2011.11.020

Method of producing graphene:
In experiment, we employed the thermal evaporator to deposit 200 nm thick Cu films on a 2 inch quartz substrate. The Cu coated quartz sample was placed in the tube furnace (Carbolite CTF 12/75/700).  The chamber was pumped into a vacuum and purged twice with nitrogen and once with hydrogen to ensure all remaining moisture and air to be removed. After the cleaning the chamber was pumped down to vacuum and injected with hydrogen up to 5.5 mBar pressure at room temperature. In the next stage, the temperature was risen up to 700 °C at the rate of 10 °C/min. Then the chamber was pumped down and the graphitization gas (1:1 H2:CH4) mixture was let into the chamber rising the pressure up to 10 mBar. At the next step, the temperature was risen up to 950 °C with rate of 10 °C/min. The maximum temperature was kept for one minute followed the cooling at the rate about 5-10 °C/min (see Fig. S1) in H2:CH4 atmosphere. At 700 °C the methane-hydrogen gas mixture was pumped out from the chamber and replaced with hydrogen at the pressure of 10 mBar. Then the chamber cooled down in overnight to room temperature.
''',

'''Author:
Dapeng Wei

Doi:
https://doi.org/10.1016/j.carbon.2012.11.026

Method of producing graphene:
Two 1 x 2 cm quartz wafers were used as substrate to grow graphene. The quartz wafers were first cleaned by ultrasonication in methanol, acetone, and DI water, and were dried by high purity N2 gas. Photoresist S-1805 (from Shipley Comp.) was diluted with a volume ratio of 1:6 in Thinner-P (from MicroChem Corp.), and was then spin-coated on the quartz wafer at 10,000 rpm. The thickness of the coated film is about 30 nm. The photoresist-coated quartz wafers were then baked for 5 min at 120oC. One coated quartz wafer was covered by another piece of quartz wafer, and then mounted on a sample stage in a vacuum chamber. Before growing graphene, the chamber was pumped and purged by high-purity N2 gas, and maintained at a pressure below 0.1 Torr. A continuous wave (CW) Nd:YAG laser with a wavelength of 532 nm was focused on the S-1805 film through the transparent quartz substrate using a lens of 150 mm focal length. The carbon atoms were decomposed from the laser heated photoresist, then dissolved in the molten quartz, and extracted to form graphene when the temperature of quartz was decreased. With our optical setup, the graphene was produced with a laser power of 2.8 W, irradiated for 3–5 min.



'''
]],

    'ground_truth' : ['The purpose of the quartz reactor in the method of producing graphene is to house the copper substrates and create the necessary conditions for the growth of graphene. It maintains an initial vacuum, allows for temperature control, and facilitates the introduction of gases for the growth process.'],

}
dataset = Dataset.from_dict(data_samples)

from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.3600, 'faithfulness': 0.5000}

###Question 2

In [ ]:
rag_query("What are the steps involved in the process of graphene nucleation and growth?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author:
Xuebin Wang

Doi:
10.1038/ncomms3905

Method of producing graphene:
Typically, 10 g glucose was mixed with 10 g ammonium salts (NH4Cl), which was then heated under a desired heating rate (4OC/min  was recommended) and finally treated at 1,350 OC for 3 h under Ar atmosphere in a tube furnace (50 cm length by 5 cm diameter). A black foam-like product, that is, (strutted graphene) SG, was collected. White granulated sugar (a kitchen-use sugar from Nissin-Sugar Co.) was also used to replace glucose to grow SG.


Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder 

In [ ]:
from datasets import Dataset

data_samples = {
    'question': ['What are the steps involved in the process of graphene nucleation and growth?'],
    'answer': ['''## Graphene Nucleation and Growth: A Step-by-Step Process

While the provided contexts offer various methods for graphene synthesis, none directly detail the fundamental steps of nucleation and growth. However, we can extrapolate those principles and combine them with general knowledge to outline the process.

**Let's use the context provided by G. Deokar (DOI: https://doi.org/10.1016/j.carbon.2015.03.017) as a basis, focusing on Chemical Vapor Deposition (CVD) on copper foil:**

**1. Precursor Adsorption and Decomposition:**

* Gaseous precursor molecules (methane in this context) are introduced into the reaction chamber containing the heated copper substrate.
* These molecules adsorb onto the copper surface.
* Upon reaching a sufficient temperature, the precursor molecules decompose, releasing carbon atoms.

**2. Surface Diffusion and Supersaturation:**

* The decomposed carbon atoms become mobile on the copper surface and diffuse across it.
* If the rate of carbon deposition exceeds the rate of desorption, a supersaturation of carbon atoms occurs on the copper surface.

**3. Nucleation:**

* Once the supersaturation reaches a critical threshold, carbon atoms start to bond together, forming small clusters or "nuclei." These nuclei act as starting points for graphene growth.
* The shape and size of these nuclei are influenced by factors like temperature, pressure, and the interaction between carbon and the copper substrate.

**4. Growth:**

* More carbon atoms diffuse across the surface, attaching to the edges of existing nuclei.
* The nuclei grow laterally into larger graphene islands.
* These islands continue to grow and eventually merge with each other, forming a continuous graphene film.

**5. Growth Termination and Defect Healing:**

* Growth continues until the precursor supply is stopped or the process conditions change.
* During the cooling phase, some defects in the graphene lattice can be healed through annealing processes.

**Factors Influencing Nucleation and Growth:**

Several factors significantly influence graphene nucleation and growth during CVD:

* **Substrate:** The choice of substrate material (copper in this case) affects the carbon atom's diffusion rate and interaction, ultimately influencing nucleation density and growth rate.
* **Temperature:** Higher temperatures enhance precursor decomposition and surface diffusion, leading to larger graphene domains.
* **Pressure:**  Pressure affects the precursor concentration near the substrate surface. Lower pressure typically promotes larger graphene grain size.
* **Precursor Gas:**  The type and flow rate of precursor gas determine the carbon supply for graphene growth.

Understanding these fundamental steps in graphene nucleation and growth is crucial for optimizing synthesis processes and tailoring graphene properties for specific applications.

'''],

    'contexts' : [['''Author:
Xuebin Wang

Doi:
10.1038/ncomms3905

Method of producing graphene:
Typically, 10 g glucose was mixed with 10 g ammonium salts (NH4Cl), which was then heated under a desired heating rate (4OC/min  was recommended) and finally treated at 1,350 OC for 3 h under Ar atmosphere in a tube furnace (50 cm length by 5 cm diameter). A black foam-like product, that is, (strutted graphene) SG, was collected. White granulated sugar (a kitchen-use sugar from Nissin-Sugar Co.) was also used to replace glucose to grow SG.
''',

'''Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and stirred for 48 h and separate filtrate and precipitate. Finally, precipitate was dried at 80oC for 24 h and characterized by using XRD and SEM-EDX, respectively.
''',

'''Author:
G. Deokar

Doi:
https://doi.org/10.1016/j.carbon.2015.03.017

Method of producing graphene:
Graphene was synthesized on polycrystalline Cu foils (50 um thick Alfa-Aesar, purity 99.9999%) in a rapid thermal processing CVD chamber, fitted with a pressure regulation device and temperature control by an optical pyrometer. The typical sample size was a few cm2 . Cu foils were pre-cleaned in acetic acid at 35 degree Celsius for 10 min in order to remove the surface oxide. The samples were immediately loaded in the CVD growth reactor and pumped to the base vacuum pressure (105 mbar). Then, the Cu foil was heated to 1060oC and held for 5 min under Ar/H2:100/5 sccm (standard cubic centimetre per minute at standard temperature and pressure) with 10 Torr chamber pressure. Afterwards, the gaseous carbon source, CH4 (20 sccm), was introduced for 5 min to carry out the graphene growth. Finally, the sample was fast cooled (30oC/s) below 700oC and further slowly to room temperature under H2 and Ar ambient. It should be noted that its total duration is only 30 min, which is much less than the process time in standard hot-wall furnaces




'''
]],

    'ground_truth' : ['The steps involved in the process of graphene nucleation and growth are as follows: (1) heating up the CVD system from room temperature to the growth temperature, (2) Cu foil annealing, (3) graphene nucleation and growth, (4) cooling down the system to room temperature.'],

}
dataset = Dataset.from_dict(data_samples)

from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.0000, 'faithfulness': 0.1026}

###Question 3

In [ ]:
rag_query("What is the process involved in the Modified Hummers' method for producing graphene?")


> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Chellachamy Anbalagan Amarnath

Doi: 
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS


Author: 


In [ ]:
from datasets import Dataset

data_samples = {
    'question': ["What is the process involved in the Modified Hummers' method for producing graphene?"],
    'answer': ['''The Modified Hummers' method is a widely used approach for synthesizing graphene oxide, which can be further reduced to obtain graphene. Here's a breakdown of the general steps involved:

**1. Oxidation of Graphite:**

   - **Pre-treatment:** Graphite powder is typically pre-treated with a strong oxidizing agent like sulfuric acid (H₂SO₄). This step helps to intercalate the graphite layers, making them more susceptible to oxidation.
   - **Oxidation:**  Potassium permanganate (KMnO₄) is then introduced as a strong oxidizing agent. The reaction is often carried out at a controlled temperature and under vigorous stirring. This step introduces oxygen-containing functional groups onto the graphite layers, forming graphite oxide.

**2. Washing and Purification:**

   - **Quenching:** The reaction is quenched by adding water, often carefully and in stages to control the exothermic reaction.
   - **Removal of Oxidants:**  Hydrogen peroxide (H₂O₂) is commonly used to remove excess KMnO₄ and manganese dioxide (MnO₂) byproducts.
   - **Washing:** Repeated washing cycles, often using dilute acid (like HCl) and deionized water, are performed to remove residual salts and impurities. This ensures a purer form of graphite oxide.

**3. Exfoliation and Reduction (to obtain Graphene):**

   - **Exfoliation:** Graphite oxide is exfoliated to obtain graphene oxide (GO).  This can be achieved through various methods, including:
     - **Sonication:** Applying ultrasonic waves to the GO dispersion.
     - **Thermal Exfoliation:** Rapid heating of GO, causing the expulsion of intercalated molecules and separating the layers.
   - **Reduction:**  GO is reduced to remove the oxygen-containing functional groups, restoring the sp² hybridized carbon structure and producing graphene. Common reduction methods include:
     - **Chemical Reduction:**  Using reducing agents like hydrazine, ascorbic acid, or hydroquinone.
     - **Thermal Reduction:**  Heating GO in an inert atmosphere or under vacuum.

**Context Selection:**

The context provided by **Baolin Xing (DOI: https://doi.org/10.1016/j.fuproc.2017.05.021)** gives a good, detailed overview of the Modified Hummers' method, specifically highlighting the use of carbonized humic acid as the starting material and the thermal exfoliation-reduction process for obtaining graphene nanosheets.

'''],

    'contexts' : [['''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS
''',

'''Author:
Baolin Xing

Doi:
https://doi.org/10.1016/j.fuproc.2017.05.021

Method of producing graphene:
2.2. Synthesis of graphene nanosheets The carbonized powder from humic acid was used as starting materials to synthesize graphene oxide by using a modified Hummers method. Briefly, 5 g carbonized powder, 30 g KMnO4 and 5 g NaNO3 were slowly added into 220 ml concentrated H2SO4 in sequence and vigorously stirred in an ice bath for 10 h. The mixture was then heated to 35 °C and maintained for 3 h with continuous stirring. The mixture was then diluted by 440 ml distilled water, and the temperature of mixture was controlled to be under 85 °C during the dilution process. After that, H2O2 (30 wt%) was added into the mixture drop by drop until the color of solution became bright yellow. The mixture was finally centrifuged (8000 rpm, 6 min) and the supernatant was discarded. The remaining solid was then successively washed with 1 M HCl solution and deionized water several times by centrifugation until the pH of the solution was neutral. The obtained solid was ultrasonicated in deionized water for 8 h, and the resultant suspension was then dried by freeze drying to obtain graphene oxide. For the thermal exfoliation-reduction of graphene oxide process, 1 g of obtained graphene oxide was placed in a quartz crucible (with a lid) and heated at 900 °C for 5 min in a muffle furnace under nitrogen atmosphere. The graphene nanosheets were collected after a natural cooling.
''',

'''Author:
Karthikeyan Krishnamoorthy

Doi:
https://doi.org/10.1016/j.ultsonch.2012.09.007

Method of producing graphene:
The graphene oxide was synthesized according to the modified Hummer’s method using graphite powder as the starting material. Briefly, graphite powder (2 g) was stirred in 98% H2SO4 (35 mL) for 2 h. KMnO4 (6 g) was gradually added to the above solution while keeping the temperature less than 20oC. The mixture was then stirred at 35–40 oC for 30 min. The resulting solution was diluted by adding 90 mL of water under vigorous stirring and a dark brown color suspension was obtained. The reaction was terminated by the addition of 150 mL of distilled water and 30% H2O2 solution (10 mL). After continuously stirred for 2 h, the mixture was washed by repeated centrifugation and filtration using 5% HCl aqueous solution in order to remove the residual metal ions. Further, the centrifugation process was repeated with distilled water until the pH of the solution becomes neutral. Finally, a brown colored precipitate of graphitic oxide was obtained by filtration and was allowed to dry under vacuum. The exfoliation of graphitic oxide into graphene oxide was achieved by ultrasound irradiation. Briefly, 0.2 g of as-synthesized graphitic oxide was redispersed in 200 mL of distilled water and allowed to sonication for 30 min which results in exfoliation of graphitic oxide into monolayers of graphene oxide (GO) and is further used for the synthesis of graphene.



'''
]],

    'ground_truth' : ["The process involved in the Modified Hummers' method for producing graphene includes the preparation of Graphene Oxide (GO) by mixing graphite, NaNO3, and H2SO4 in a round-bottom flask with magnetic stirring. KMnO4 is added to the reaction system and kept at 0-5 °C for 1 hour, followed by heating at 35 °C for 2 hours. The mixture is then diluted with deionized water and reacted at 98 °C for 45 minutes. Finally, deionized water and H2O2 are added to the suspension, and after centrifugation and drying, Graphene Oxide (GO) is obtained."],

}
dataset = Dataset.from_dict(data_samples)

from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.0000, 'factual_correctness': 0.0800, 'faithfulness': 0.6333}

###Question 4

In [ ]:
rag_query("What is the method of producing graphene oxide?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Hareema Saleem

Doi:  
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was c

In [ ]:
from datasets import Dataset

data_samples = {
    'question': ['What is the method of producing graphene oxide?'],
    'answer': ['''While all the contexts describe methods of producing graphene, they all utilize graphene oxide as a precursor material.  The actual method of producing graphene *oxide* is best described in the context provided by **Kaveri Satheesh**.

Here's a breakdown of the process as described in that context:

**1. Oxidation of Graphite:**

* **Materials:** Graphite powder, concentrated H2SO4, NaNO3, KMnO4, water, 30% H2O2, 10% HCl, ethanol.
* **Process:**
    * Graphite powder is mixed with H2SO4 and NaNO3 in an ice bath.
    * KMnO4 is slowly added, keeping the temperature below 10°C.
    * The mixture is heated to 35°C for 1 hour.
    * Water is added, further raising the temperature.
    * The mixture is heated to 98°C for 2 hours.
    * The reaction is stopped by adding water and H2O2.
    * The mixture is filtered and washed multiple times.
    * The resulting solid is dried in a vacuum oven.

**2. Product:**  The process yields graphene oxide (GO) as a dry brown powder.

**Key takeaway:** This method uses a modified Hummer's method, a widely used approach for synthesizing graphene oxide from graphite.
'''],

    'contexts' : [['''Author:
Hareema Saleem

Doi:
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was converted into mustard color. Washed the mixture with 1 M HCl and distilled water till the pH of supernatant liquid reached 7

2.2. Exfoliation of graphene oxide (GO):
To exfoliate the GO, sonicated the GO dispersion under normal condition for 30 min. The homogeneous brown dispersion was produced which was utilized for reduction. Dried the solution in the vacuum oven at 40 oC  for 24 h 2.3. Thermal reduction of graphene oxide The GO was annealed in a Nabertherm Gmbh N17/HR-400V Muffle heating furnace at 500 oC at a heating rate of 2 oC /min for 2 h and allowed to cool in furnace till the temperature decreased till 50 ± 5 oC. The GO was completely converted into reduced graphene oxide (rGO)
''',

'''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS
''',

'''Author:
Kaveri Satheesh

Doi:
https://doi.org/10.1016/j.matlet.2013.09.044

Method of producing graphene:
Synthesis of GO: Graphite Oxide (GO) was synthesized from graphite powder by using Hummers method. 2 g of graphite powder was added to a mixture of 46 ml of concentrated H2SO4 and 1 g of NaNO3 in an ice bath. Then 6 g of KMnO4 was added slowly to the suspension under stirring to avoid any violent or explosive reactions and the temperature of the mixture was kept below 10oC by cooling. After adding the KMnO4 to the mixture, the dark green suspension was removed from the ice bath and the temperature was raised to 35 oC and maintained 1 h, which resulted in a thick paste. Then the mixture was diluted by 92 ml of water. Because of the addition of water in concentrated sulfuric acid medium a large amount of heat was released. Then the temperature was raised to 98 oC and maintained for 2 h. The reaction was terminated by the addition of 280 ml of water and 30% H2O2, resulting in brown mixture. Then the mixture was filtered and washed with 10% HCl, ethanol and water. The resulting solid was dried in a vacuum oven at 60 oC for 12 h. Then GO was obtained in the form of dry brown powder.

Synthesis of graphene sheets: Typically, 100 mg of GO was dispersed in 100 ml of water through ultrasonication for 1 h to yield GO dispersion. 0.5 mol of thiourea was added to the dispersion under stirring at 95 oC for 12 h. With the reduction process, the brown color of GO dispersion turns into black color. Finally the black color product (rGO) was washed with water and ethanol. The filtered product was dried in a vacuum oven at 60 oC for 12 h.


'''
]],

    'ground_truth' : ['Graphene oxide is produced through the Hummers method, which involves pre-oxidizing natural graphite by mixing it with concentrated sulfuric acid, potassium persulfate, and phosphorus pentoxide. The mixture is then heated, diluted with water, and stirred. The product is recovered, washed, and dried. The pre-oxidized graphite is then dispersed in sulfuric acid, and potassium permanganate is gradually added. The dispersion is incubated and diluted with water. To dissolve the KMnO4, hydrogen peroxide is added, and the mixture is stirred. The suspension is filtered, washed, and dried to obtain graphite oxide. The graphite oxide is then exfoliated in water using an ultrasonic bath, and reduced graphene oxide is obtained through thermal reduction in an Ar/5% H2 atmosphere.'],

}
dataset = Dataset.from_dict(data_samples)

from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.6250, 'factual_correctness': 0.0000, 'faithfulness': 0.9000}

###Question 5

In [ ]:
rag_query("What is the process of graphite foil pretreatment in the production of graphene?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author:
Mustafa K. Bayazit

Doi:
10.1021/acsami.1c03906

Method of producing graphene:
In a typical procedure, commercial graphite ﬂakes (2 g) were added to a sealed vial containing Br2 /solvent solution and then left for 2 h for intercalation at room temperature. Similarly, 1-, 3-, and 7-day treatment terms were also used to optimize the pretreatment process. Water, chloroform, and n-decane were selected as solvents. P-graphite ﬂakes were altered through ﬁlter paper and washed with fresh chloroform to remove the remaining Br2 molecules and then was transferred into a clean vial and dried at room temperature in a fume cu

In [ ]:
data_samples = {
    'question': ['What is the process of graphite foil pretreatment in the production of graphene?'],
    'answer': ['''The most relevant context to answer this question is from **Sheng Yang's work (DOI: 10.1021/jacs.5b09000).**

Here's the pretreatment process they describe:

* **Alternating Liquid Nitrogen and Ethanol Soaking:** Graphite foil is repeatedly submerged in liquid nitrogen, causing rapid cooling and potential weakening of interlayer forces.  This is followed by soaking in absolute ethanol, which likely infiltrates between the expanded layers. The rapid transition between extreme temperatures and ethanol solidification contributes to the expansion and separation of graphite layers, priming them for further exfoliation.

**Let's break down why this context is relevant and how the process works:**

* **Focus on Graphite Foil:** The question specifically asks about pretreating **graphite foil**, which is the starting material in Yang's work. The other contexts use graphite flakes, making them less relevant.
* **Facilitating Exfoliation:** The pretreatment aims to make the graphite layers easier to separate, which is the essence of graphene production. Yang's method achieves this through a physical process of expansion and weakening of the interlayer forces.

**In summary, Yang's pretreatment process utilizes the extreme temperature differences between liquid nitrogen and ethanol to expand the graphite foil, making it more susceptible to exfoliation in the subsequent electrochemical process.**

'''],

    'contexts' : [['''Author:
Mustafa K. Bayazit

Doi:
10.1021/acsami.1c03906

Method of producing graphene:
In a typical procedure, commercial graphite ﬂakes (2 g) were added to a sealed vial containing Br2 /solvent solution and then left for 2 h for intercalation at room temperature. Similarly, 1-, 3-, and 7-day treatment terms were also used to optimize the pretreatment process. Water, chloroform, and n-decane were selected as solvents. P-graphite ﬂakes were altered through ﬁlter paper and washed with fresh chloroform to remove the remaining Br2 molecules and then was transferred into a clean vial and dried at room temperature in a fume cupboard. Dried samples were labelled as the P-graphite and stored at room temperature in a sealed glass vessel. 20−100 mg of the P-graphite was transferred into a sealed glass tube and irradiated by a microwave irradiation for about 10 s, using a CEM microwave ﬁtted with an Infrared (IR) temperature sensor. The temperature of the solid reaction material was recorded as 360 ± 20°C. The reaction medium was cooled to 50 °C, and the glass tube was opened in a fume cupboard. Highly exfoliated solid graphitic material was obtained and labelled as EG.


Author:
Sheng Yang

Doi:
10.1021/jacs.5b09000

Method of producing graphene:
Graphite exfoliation was conducted in a home-built setup that used rolled graphite foils as working electrodes and platinum foils as counter electrodes. An aqueous solution containing ammonium sulphate with TEMPO was tested as an electrolyte. Before exfoliation, the graphite foil was pretreated by alternately soaking it in liquid nitrogen and absolute ethanol. During this process, violent nitrogen evolution and ethanol solidification promoted the expansion of the graphite foils. The dilation was extremely rapid (within 10 s) and yielded a drastic volume increase at the graphite boundary (∼10 times), which facilitated the expansion of the graphite layers, as did the following anion intercalation steps. When the graphite anodes were immersed in the electrolyte, by applying a static bias of 10 V, they detached into small pieces in less than 10 s, spreading on top of the electrolyte and/or on the bottom of the electrochemical cell. After filtration and copious washing, these exfoliated graphene sheets were collected and then dispersed in DMF via mild sonication. A stable dispersion with a concentration of ∼0.3 mg/mL was obtained that was stable for 2 weeks without apparent aggregation.


Author:
F.S. Al-Hazmi

Doi:
https://doi.org/10.1016/j.synthmet.2014.12.028

Method of producing graphene:
5 g of graphite flakes (thickness ∼0.1 μm) was first immersed into 30 ml of ice cooled glutaric acid over night. The pretreated graphite flakes were filtered and left to dry at ambient conditions. The pretreated graphite subjected to a thermal shock at 700 °C for 1 min to expand the graphite. 500 mg of the expanded graphite flakes were dispersed in 50 ml methanol and sonicated for 10 min to ensure the complete dispersion of pretreated graphite in the methanol solution. The graphite solution was introduced into a Teflon autoclave with capacity 80 cm3 and put into microwave oven at power of 800 W and temperature 200 °C for 30 min. After this time, the reaction was terminated by put the Teflon autoclave into ice cooled bath. The supernatant was collected by centrifuging and washed several times with deionized water and HCl (10%). The graphene sheets were dried under vacuum over night at 70 °C.
'''
]],

    'ground_truth' : ['The graphite foil is pretreated by alternately soaking it in liquid nitrogen and absolute ethanol. This process promotes the expansion of the graphite foils through violent nitrogen evolution and ethanol solidification. The expansion of the graphite layers is facilitated by the drastic volume increase at the graphite boundary, as well as the following anion intercalation steps.'],

}
dataset = Dataset.from_dict(data_samples)
dataset
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.1400, 'faithfulness': 0.3182}

###Question 6

In [ ]:
rag_query("How were the Ni substrates heated and maintained in the production of Single Layer Graphene and Few Layer Graphene?")


> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Zhen-Yu Juang

Doi: 
https://doi.org/10.1016/j.carbon.2010.05.001

Method of producing graphene:
Single Layer Graphene and Few Layer Graphene were synthesized using Ni foil (30 um in thickness, The Nilaco Corp.) as the substrate. After the Ni substrates were loaded into a quartz tubular furnace, the samples were heated to the process temperature of 900oC and maintained for 10 min under a H2/Ar (10 and 400 sccm, respectively) atmosphere. A CH4/H2/Ar gas mixture (10, 10 and 400 sccm, respectively) was then introduced into the quartz tube for 10 min, and the furnace was cooled to room temperature with fast (3 oC/s)

In [ ]:
data_samples = {
    'question': ['How were the Ni substrates heated and maintained in the production of Single Layer Graphene and Few Layer Graphene?'],
    'answer': ['''The most relevant context to answer this question is the first one provided:

**Author:** Zhen-Yu Juang

**DOI:** https://doi.org/10.1016/j.carbon.2010.05.001

In this context, the authors describe heating the Ni substrates in a **quartz tubular furnace**. Here's a breakdown of the heating process:

1. **Loading:** The Ni substrates are placed inside the quartz tubular furnace.
2. **Heating:** The furnace is heated to a process temperature of **900 °C**.
3. **Atmosphere:** This heating occurs under a continuous flow of a **H2/Ar gas mixture** (10 sccm and 400 sccm, respectively).
4. **Holding Time:** The temperature is maintained at 900 °C for **10 minutes**.
5. **Pressure:** Throughout the entire heating process, the pressure inside the furnace is kept constant at **750 Torr** using an advanced pressure control system.

This context specifically answers your question about the heating method and conditions used for producing Single Layer Graphene and Few Layer Graphene on Ni substrates.

'''],

    'contexts' : [['''Author:
Zhen-Yu Juang

Doi:
https://doi.org/10.1016/j.carbon.2010.05.001

Method of producing graphene:
Single Layer Graphene and Few Layer Graphene were synthesized using Ni foil (30 um in thickness, The Nilaco Corp.) as the substrate. After the Ni substrates were loaded into a quartz tubular furnace, the samples were heated to the process temperature of 900oC and maintained for 10 min under a H2/Ar (10 and 400 sccm, respectively) atmosphere. A CH4/H2/Ar gas mixture (10, 10 and 400 sccm, respectively) was then introduced into the quartz tube for 10 min, and the furnace was cooled to room temperature with fast (3 oC/s) and slow (0.3 oC/s) cooling rates. The gas mixture during cooling is the same as heating stage. The pressure was maintained at 750 Torr during whole process using advanced pressure control system (APC, MKS Instruments).
''',

'''Author:
Zhen-Yu Juang

Doi:
https://doi.org/10.1016/j.carbon.2009.03.051

Method of producing graphene:
In our process, single-crystalline 6H–SiC(0 0 0 1) substrates (SiCrystal AG) and 3C–SiC coated Si substrates were coated with 200 nm Ni (patterned and non-patterned) thin films via electron beam evaporation. After the substrates were put on the top of the flat heater in a vacuum chamber, the pressure was lowered to 10−7 Torr. The sample was subsequently heated to the process temperature and immediately cooled to room temperature by turning off the heater. The initial cooling rate was around 10–20 °C/s. Note that the thermal couple was in direct contact with the surface of the flat heater.
''',

'''Author:
L. Huang

DOI:
https://doi-org.remotexs.ntu.edu.sg/10.1016/j.carbon.2011.09.012

Method of producing graphene:
The process was carried out at home-made cold-wall reactor apparatus consisting of a RTP system (RTP1000, Ecopia) and a modified gas supply. Prior to CVD, the Ni foil (thickness: 25 um, purity: 99.99%) was placed in a reaction ion etching system   (etch time 30 min, RF power 100 W, Ar gas flow 100 sccm, gas pressure 2 Pa) to create a fresh Ni surface. The Ar plasma treated Ni foils was immediately loaded onto the PG (Pyrolytic Graphite)/PBN (Pyrolytic Boron Nitride) heater stage. The reaction chamber was pumped down to 3 mTorr using a rotary vacuum pump. The Ni foil was heated up to 1000 degree Celsius at a 100 degree Celsius /min heating rate, and maintained for 30 min in the N2/H2 atmosphere (N2:H2 = 700:100 sccm, total pressure 4.5 Torr). Subsequently, the CVD growth of graphene was triggered by the addition of CH4 (8 sccm) along with N2 (700 sccm) and H2 (100 sccm) to maintain 20 Torr of the chamber pressure. After CVD growth, the samples were cooled down from 1000 to 500oC with a consistent cooling rate of 200oC /min, under a flow of N2 (700 sccm). After CVD growth, the as-grown graphene films were transferred to different target substrates using the following method. First, a thin layer of poly(methyl methacrylate) (PMMA) was spin-coated onto the as-grown graphene at 500 rpm for 5 s first, then 2500 rpm for 20 s. The PMMA/graphene/Ni foil was cured at 180 oC for 1 min, and subsequently, the rear side of the Ni foil was polished to remove the coatings. The graphene/PMMA film was released from the Ni foil by chemical etching of Ni with a FeCl3 (1 M) solution over a period of 24 h, followed by rinsing with de-ionized water. The floating PMMA supported-graphene was transferred onto arbitrary substrates such as a SiO2/Si substrate and a flexible transparent PET substrate (0.125 mm thick film). Finally, PMMA was dissolved by acetone. Produced mono-layer graphene


'''
]],

    'ground_truth' : ['''The Ni substrates were heated to the process temperature of 900oC and maintained for 10 min under a H2/Ar (10 and 400 sccm, respectively) atmosphere.
'''],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.4600, 'faithfulness': 1.0000}

###Question 7

In [ ]:
rag_query("What is the purpose of plasma cleaning in graphene production using a CVD apparatus with microwave launchers and slot antennas?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Takatoshi Yamada

Doi: 
10.1088/0022-3727/46/6/063001

Method of producing graphene:
Our CVD apparatus is equipped with four parallel microwave launchers having five slot antennas in each launcher. It has a substrate holder (400 mm × 600 mm), which enables the reciprocating motion of the substrate in order to deposit graphene on a large area with high uniformity. Graphene depositions were carried out by a two-step process. The first step was the plasma cleaning of the substrate surface, and the second step was graphene deposition. The substrates were mounted on the sample holder. The CVD chamber was evacuated us

In [ ]:
data_samples = {
    'question': ['What is the purpose of plasma cleaning in graphene production using a CVD apparatus with microwave launchers and slot antennas?'],
    'answer': ['''The most relevant context to answer this question is from Takatoshi Yamada's work.

**Purpose of Plasma Cleaning:**

In Takatoshi Yamada's research on graphene production using a CVD apparatus with microwave launchers and slot antennas, plasma cleaning plays a crucial role in preparing the substrate surface for graphene deposition.

* **Removal of Contaminants and Oxides:**  The plasma cleaning process, utilizing an Ar/H2 plasma, effectively removes surface contaminants and oxides from the substrate. This is essential because contaminants can interfere with the growth of uniform, high-quality graphene layers.

* **Enhanced Graphene Quality:** By ensuring a pristine substrate surface, plasma cleaning promotes the growth of graphene with improved properties.

**Important Considerations:**

* The specific plasma conditions (gas mixture, pressure, microwave power) are critical for successful plasma cleaning. These parameters are optimized to effectively remove contaminants without damaging the substrate.
* Yamada's work highlights that plasma cleaning can negate the need for high-temperature heat treatments typically used to prepare Cu substrates for graphene growth. This suggests plasma cleaning is a more efficient and potentially less damaging pretreatment method.

'''],

    'contexts' : [['''Author:
Takatoshi Yamada

Doi:
10.1088/0022-3727/46/6/063001

Method of producing graphene:
Our CVD apparatus is equipped with four parallel microwave launchers having five slot antennas in each launcher. It has a substrate holder (400 mm × 600 mm), which enables the reciprocating motion of the substrate in order to deposit graphene on a large area with high uniformity. Graphene depositions were carried out by a two-step process. The first step was the plasma cleaning of the substrate surface, and the second step was graphene deposition. The substrates were mounted on the sample holder. The CVD chamber was evacuated using a turbo molecular pump. For plasma cleaning of the substrate a gas mixture of Ar and H2 was used, and the duration of the cleaning process was 20 min. After that, the chamber was evacuated once, and the graphene deposition process was performed using a CH4/H2/Ar plasma. Typical plasma conditions of the cleaning and deposition processes are listed as such: “ CH4/Ar/H2 flow rate (sccm) 30/20/50; Pressure (Pa) 30 ;Microwave power (kW) 16 ;Web flow rate (mm s−1) 5–10”.
The substrates used in this study were rolled Cu foils (33µm in thickness) and rolled aluminium (Al) foils (12µm in thickness). The average surface roughness of the Cu substrate measured by AFM was about 1 nm and typical domain sizes of the Cu substrate area were in the range between 100 and 400µm. For graphene deposition by thermal CVD, a high-temperature heat treatment at around 1000 ◦C before graphene deposition was important to deposit high-quality graphene since the heat treatment leads to a highly (1 1 1) oriented Cu surface and enlarges the grain size. However, such phenomena were not observed after our plasma treatment in this study. Our plasma treatment enabled us to remove the contaminations and the oxides on the substrate surface. The temperature was measured by a thermo-couple. The substrate temperature was raised up to 245 ◦C by the plasma exposure during the pretreatment. The maximum substrate temperature during the deposition of graphene was about 320 ◦C.
''',

'''Author:
Lam Van Nang

Doi:
10.1149/2.082204jes

Method of producing graphene:
To investigate the rf plasma effect, graphene growth was carried out with a diluted CH4 gas, in which graphene was not formed without applying plasma. The CH4 gas of 1 SCCM (denoting cubic centimeter per minute at STP) and the mixed gases of 10% H2 and 90% Ar (forming gas) of 100 SCCM were introduced in a hot-wall tubular reaction chamber. The Cu foil (Alfa Aesar) was pre-treated with an acetic acid solution at 60oC for 10 min, followed by washing with de-ionized water. The foil was heated to a graphene synthesis temperature (950 oC) and annealed for 10 min with forming gas flow (100 SCCM) to remove native oxides from the Cu foil and to enlarge its grains. The synthesis pressure was fixed at 1 Torr. The synthesis time and rf plasma power is 5min and 200 W, respectively. The synthesized graphene on Cu was transferred on SiO2/Si or glass substrate for various characterizations. First, a 500 nm layer of poly-methyl methacrylate (PMMA) was spin coated on the surface of the graphene film to serve as support. The graphene layer at the back side of the Cu foil was then etched off using oxygen plasma. After the etching of the Cu foil with FeCl3 aqueous solution at 40 oC, the PMMA/graphene was transferred to a target substrate. The graphene on the SiO2/Si or glass substrate was achieved by dissolving the PMMA film using acetone
''',

'''Author:
Teresa Campo

Doi:
https://doi.org/10.3844/ajeassp.2016.574.583

Method of producing graphene:
After cleaning, copper substrates were placed into a quartz boat and introduced in a cylindrical quartz reactor (inner diameter of 25 mm and 1 m-length) within a tube furnace. The quartz reactor was maintained at an initial vacuum of 10^-2 Torr using a mechanical pump. The temperature was risen to 950°C, at a rate of 30°C/min (Guermoune et al., 2011). Samples were maintained at this temperature for 26 min, using an H2 constant flow of 10 sccm to remove possible organic impurities. After the first 20 min of this step, a flow mixture of N2 gas and ethanol vapor (30 sccm), previously generated by an overpressure of N2, was introduced into the reactor. In this moment, a blue plasma discharge was generated. At the same time, the reactor pressure increases to 102 Torr during a few seconds until the pressure stabilizes at 1 Torr. Once the growth time finishes, a cooling process takes place. During the first 30 min of the cooling step, a constant flow of H2 10 sccm is maintained and, after that, an Ar flow of 300 sccm is introduced into the reactor.



'''
]],

    'ground_truth' : ['The purpose of plasma cleaning in graphene production using a CVD apparatus with microwave launchers and slot antennas is to remove contaminations and oxides on the substrate surface.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.2000, 'faithfulness': 0.6667}

### To check the change in score without extra information

In [ ]:
data_samples = {
    'question': ['What is the purpose of plasma cleaning in graphene production using a CVD apparatus with microwave launchers and slot antennas?'],
    'answer': [''' **Removal of Contaminants and Oxides:**  The plasma cleaning process, utilizing an Ar/H2 plasma, effectively removes surface contaminants and oxides from the substrate.
'''],

    'contexts' : [['''Author:
Takatoshi Yamada

Doi:
10.1088/0022-3727/46/6/063001

Method of producing graphene:
Our CVD apparatus is equipped with four parallel microwave launchers having five slot antennas in each launcher. It has a substrate holder (400 mm × 600 mm), which enables the reciprocating motion of the substrate in order to deposit graphene on a large area with high uniformity. Graphene depositions were carried out by a two-step process. The first step was the plasma cleaning of the substrate surface, and the second step was graphene deposition. The substrates were mounted on the sample holder. The CVD chamber was evacuated using a turbo molecular pump. For plasma cleaning of the substrate a gas mixture of Ar and H2 was used, and the duration of the cleaning process was 20 min. After that, the chamber was evacuated once, and the graphene deposition process was performed using a CH4/H2/Ar plasma. Typical plasma conditions of the cleaning and deposition processes are listed as such: “ CH4/Ar/H2 flow rate (sccm) 30/20/50; Pressure (Pa) 30 ;Microwave power (kW) 16 ;Web flow rate (mm s−1) 5–10”.
The substrates used in this study were rolled Cu foils (33µm in thickness) and rolled aluminium (Al) foils (12µm in thickness). The average surface roughness of the Cu substrate measured by AFM was about 1 nm and typical domain sizes of the Cu substrate area were in the range between 100 and 400µm. For graphene deposition by thermal CVD, a high-temperature heat treatment at around 1000 ◦C before graphene deposition was important to deposit high-quality graphene since the heat treatment leads to a highly (1 1 1) oriented Cu surface and enlarges the grain size. However, such phenomena were not observed after our plasma treatment in this study. Our plasma treatment enabled us to remove the contaminations and the oxides on the substrate surface. The temperature was measured by a thermo-couple. The substrate temperature was raised up to 245 ◦C by the plasma exposure during the pretreatment. The maximum substrate temperature during the deposition of graphene was about 320 ◦C.
''',

'''Author:
Lam Van Nang

Doi:
10.1149/2.082204jes

Method of producing graphene:
To investigate the rf plasma effect, graphene growth was carried out with a diluted CH4 gas, in which graphene was not formed without applying plasma. The CH4 gas of 1 SCCM (denoting cubic centimeter per minute at STP) and the mixed gases of 10% H2 and 90% Ar (forming gas) of 100 SCCM were introduced in a hot-wall tubular reaction chamber. The Cu foil (Alfa Aesar) was pre-treated with an acetic acid solution at 60oC for 10 min, followed by washing with de-ionized water. The foil was heated to a graphene synthesis temperature (950 oC) and annealed for 10 min with forming gas flow (100 SCCM) to remove native oxides from the Cu foil and to enlarge its grains. The synthesis pressure was fixed at 1 Torr. The synthesis time and rf plasma power is 5min and 200 W, respectively. The synthesized graphene on Cu was transferred on SiO2/Si or glass substrate for various characterizations. First, a 500 nm layer of poly-methyl methacrylate (PMMA) was spin coated on the surface of the graphene film to serve as support. The graphene layer at the back side of the Cu foil was then etched off using oxygen plasma. After the etching of the Cu foil with FeCl3 aqueous solution at 40 oC, the PMMA/graphene was transferred to a target substrate. The graphene on the SiO2/Si or glass substrate was achieved by dissolving the PMMA film using acetone
''',

'''Author:
Teresa Campo

Doi:
https://doi.org/10.3844/ajeassp.2016.574.583

Method of producing graphene:
After cleaning, copper substrates were placed into a quartz boat and introduced in a cylindrical quartz reactor (inner diameter of 25 mm and 1 m-length) within a tube furnace. The quartz reactor was maintained at an initial vacuum of 10^-2 Torr using a mechanical pump. The temperature was risen to 950°C, at a rate of 30°C/min (Guermoune et al., 2011). Samples were maintained at this temperature for 26 min, using an H2 constant flow of 10 sccm to remove possible organic impurities. After the first 20 min of this step, a flow mixture of N2 gas and ethanol vapor (30 sccm), previously generated by an overpressure of N2, was introduced into the reactor. In this moment, a blue plasma discharge was generated. At the same time, the reactor pressure increases to 102 Torr during a few seconds until the pressure stabilizes at 1 Torr. Once the growth time finishes, a cooling process takes place. During the first 30 min of the cooling step, a constant flow of H2 10 sccm is maintained and, after that, an Ar flow of 300 sccm is introduced into the reactor.



'''
]],

    'ground_truth' : ['The purpose of plasma cleaning in graphene production using a CVD apparatus with microwave launchers and slot antennas is to remove contaminations and oxides on the substrate surface.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.5700, 'faithfulness': 1.0000}

###Question 8

In [ ]:
rag_query("What's the process for producing reduced graphene oxide using chemical exfoliation and thermal reduction?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Hareema Saleem

Doi:  
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was c

In [ ]:
data_samples = {
    'question': ["What's the process for producing reduced graphene oxide using chemical exfoliation and thermal reduction?"],
    'answer': ['''The process described by **Hareema Saleem** (https://doi.org/10.1016/j.matchemphys.2017.10.020) provides a clear and concise method for producing reduced graphene oxide using chemical exfoliation and thermal reduction:

**1. Chemical Exfoliation:**

* **Oxidation:** Graphite flakes are oxidized using a strong acid mixture (H2SO4/H3PO4) and KMnO4. This step increases the interlayer spacing of graphite by introducing oxygen-containing functional groups, making it easier to separate into individual layers.
* **Washing:** The resulting graphite oxide is washed with HCl and distilled water to remove residual acids and impurities, achieving a neutral pH.
* **Exfoliation:** The washed graphite oxide is sonicated, breaking down the material into individual graphene oxide (GO) sheets dispersed in the solution.
* **Drying:** The GO dispersion is dried in a vacuum oven, yielding GO powder.

**2. Thermal Reduction:**

* **Annealing:** The dried GO is heated to a high temperature (500°C) in a furnace with a controlled atmosphere (inert or reducing, such as nitrogen). This process removes most of the oxygen-containing functional groups from the GO, restoring the conjugated structure and producing reduced graphene oxide (rGO).
* **Cooling:** The rGO is allowed to cool down slowly in the furnace to prevent thermal shock and potential damage to the graphene structure.

This combined process of chemical exfoliation and thermal reduction offers a scalable approach for producing rGO, which finds applications in various fields due to its unique properties.

'''],

    'contexts' : [['''Author:
Hareema Saleem

Doi:
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was converted into mustard color. Washed the mixture with 1 M HCl and distilled water till the pH of supernatant liquid reached 7

2.2. Exfoliation of graphene oxide (GO):
To exfoliate the GO, sonicated the GO dispersion under normal condition for 30 min. The homogeneous brown dispersion was produced which was utilized for reduction. Dried the solution in the vacuum oven at 40 oC  for 24 h 2.3. Thermal reduction of graphene oxide The GO was annealed in a Nabertherm Gmbh N17/HR-400V Muffle heating furnace at 500 oC at a heating rate of 2 oC /min for 2 h and allowed to cool in furnace till the temperature decreased till 50 ± 5 oC. The GO was completely converted into reduced graphene oxide (rGO)''',


'''Author:
Xin Tong

DOI:
https://doi.org/10.1016/j.jssc.2011.03.004

Method of producing graphene:
Oxidation of graphite: firstly, 1 g graphite powders (the mean diameter of 18 mm provided by Mei Ge Electrical Carbon Co., Ltd. China) were poured into the mixture of 92 mL concentrated sulfuric acid (95–98%) and 24 mL nitric acid (65%) in a 500 mL flask(under ice bath). After the suspended solution was strongly stirred at 0oC for 15 min. Then 6 g KMnO4 (AR) was gradually added in the above mixture solution. The suspended solution was stirred continuously for 1 h. We called this process low temperature oxidation. Subsequently, in order to increase the oxidation degree of the graphite, the mixture was heated to 80 oC for 30 min, and then 92 mL of deionized water was slowly added in. The suspension was stirred at 80 oC for 30 min again. After that, 5 mL H2O2 (30%) was added into the solution to reduce the residual permanganate. On completion of the reaction, the mixture was washed with 5% solution of HCl and water until sulfate could not be detected with BaCl2. Finally, the GO powders were dried at 80 oC for 24 h. The dosage of oxidizer was controlled to produce oxidized graphite of different degrees. 2.1.2. Expansion of GO In this step, the obtained GO powders (400 mg) were loaded in a quartz boat. The quartz boat was placed into a 25-mm internal diameter; 1-m long quartz tube. After the sample was flushed with argon for 30 min (the velocity of argon is 100 mL/min ), the quartz tube was heated to 300 oC for 1 h (at heating rates of 15 oC/min) in tube furnace under argon flow. 2.1.3. Reduction of EG Graphene sheets obtained from thermal expansion contain residual H and OH groups. Therefore, to remove H and OH groups, the as-prepared EG were reduced in a tube furnace by H2 (the gas flow is consisted of H2 (40 mL/min) and argon (60 mL/min)) for 2 h at 500 oC. After the GS was dispersed into ethanol, to remove a small amount of precipitate, post-treatment of sonication and centrifugation is used and then the desired graphene sheets are collected for further characterizations.''',


'''Author:
Qiqi Zhuo

Doi:
https://doi.org/10.1016/j.carbon.2012.10.014

Method of producing graphene:
In a typical experiment, firstly, 5 mg of the as-prepared GO were dispersed into 50 mL DI water under mild ultrasonic treatment to obtain a uniform GO aqueous solution (0.1 mg/ mL). Then, 10 mL GO solution (1 mg GO) was taken out to join a 20-mL vial, and 100ul of 1 mM metal salts and 10 mg NaBH4 were added into the solution. After 1 h reaction, a suspension solution was obtained. The product was collected by centrifugation and washed by 3% HCl and DI water for three times, respectively. After washing, the metal solution can be concentrated and then be used as the catalyst again to the reduction of GO. In a typical scale-up experiment, 1 L GO solution (100 mg GO) was reduced by the same method to obtain 72 mg r-GO



'''
]],

    'ground_truth' : ['The process for producing reduced graphene oxide using chemical exfoliation and thermal reduction involves oxidizing graphite flakes with a mixture of concentrated H2SO4/H3PO4 and KMnO4. The mixture is stirred at 35-40oC for 4 to 5 h, then heated and stirred at 50 oC for 12 h. After cooling, distilled water and hydrogen peroxide are added to complete the reaction. The mixture is washed with 1 M HCl and distilled water. To exfoliate the graphene oxide (GO), the GO dispersion is sonicated for 30 min. The solution is then dried in a vacuum oven at 40 oC for 24 h. Finally, the GO is annealed in a heating furnace at 500 oC for 2 h and allowed to cool. This process converts the GO into reduced graphene oxide (rGO).'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.2400, 'faithfulness': 0.4286}

###Question 9

In [ ]:
rag_query("What was the dose rate used in the irradiation process for producing graphene?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Zhiyong Wang

Doi: 
10.1088/0957-4484/21/17/175602

Method of producing graphene:
The electric arc oven for synthesis of graphene mainly comprises two electrodes and a steel chamber cooled by water. The cathode and anode were graphite rods (purity: 99.99%) with diameters of 8 and 6 mm, respectively. The current in the discharge process was maintained at 100 A. Initial air pressures from 400 to 1000 Torr were employed to investigate the influence of gas pressure. The pressure in the oven was increased to a certain level during the discharge process because of the heating effect. A 10 cm long anode graphite rod wa

In [ ]:
data_samples = {
    'question': ["What was the dose rate used in the irradiation process for producing graphene?"],
    'answer': ['''The provided contexts do not describe the use of irradiation for graphene production. Therefore, I cannot answer what dose rate was used. Please note that irradiation is not a commonly used method for graphene synthesis.


'''],

    'contexts' : [['''Author:
Zhiyong Wang

Doi:
10.1088/0957-4484/21/17/175602

Method of producing graphene:
The electric arc oven for synthesis of graphene mainly comprises two electrodes and a steel chamber cooled by water. The cathode and anode were graphite rods (purity: 99.99%) with diameters of 8 and 6 mm, respectively. The current in the discharge process was maintained at 100 A. Initial air pressures from 400 to 1000 Torr were employed to investigate the influence of gas pressure. The pressure in the oven was increased to a certain level during the discharge process because of the heating effect. A 10 cm long anode graphite rod was consumed in a period of ~10 min. After the consumption of the anode, flue products deposited on the inner wall of the chamber were collected. Several tens of grams of products were obtained per day. Heat treatment in air was performed to purify the products. The sample was heated in air from
''',

'''Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and stirred for 48 h and separate filtrate and precipitate. Finally, precipitate was dried at 80oC for 24 h and characterized by using XRD and SEM-EDX, respectively.
''',

'''Author:
Bing Deng

Doi:
10.1002/adma.201800996

Method of producing graphene:
30 in. Cu foil was wrapped onto an inner quartz tube with a diameter of ≈7.5 in., which was then loaded inside an outer 8 in. quartz tube. Graphene growth was conducted at 1000 °C using LPCVD after annealing in a H2 atmosphere. This process allowed the radial inhomogeneity in the reaction temperature and gas flow to be minimized. Very high-quality monolayer graphene was produced with a transmittance of 97.4% and sheet resistance as low as ≈125ohm−1.

'''
]],

    'ground_truth' : ['The dose rate used in the irradiation process for producing graphene was 0.24 kGy h−1.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.0000, 'factual_correctness': 0.0000, 'faithfulness': 0.6667}

### Question 10

In [ ]:
rag_query("What is the process for producing graphene using the sonochemical method, including pH adjustments and chemical addition?")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Karthikeyan Krishnamoorthy

Doi: 
https://doi.org/10.1016/j.ultsonch.2012.09.007

Method of producing graphene:
The as-synthesized GO nanosheets were used as the starting precursor for the synthesis of graphene via sonochemical method. Briefly, the pH of the as exfoliated GO solution (0.2 g in 200 mL) was adjusted to reach 10 by NaOH solution followed by the addition of 2 mL of hydrazine and the entire reaction is subjected to ultrasound irradiation for 2 h. The sonication reaction was conducted without any cooling, so that a temperature of about 60 ± 5oC was reached at the end of the reaction. The experiment is

In [ ]:
data_samples = {
    'question': ["What is the process for producing graphene using the sonochemical method, including pH adjustments and chemical addition?"],
    'answer': ['''The following describes the process for producing graphene using the sonochemical method, based on the context provided by Karthikeyan Krishnamoorthy:

**Context:**

* **Author:** Karthikeyan Krishnamoorthy
* **DOI:** https://doi.org/10.1016/j.ultsonch.2012.09.007
* **Method of producing graphene:** Sonochemical reduction of graphene oxide.

**Process:**

1. **Preparation of Graphene Oxide (GO) Solution:**  Start with a previously synthesized graphene oxide solution (e.g., 0.2 g GO in 200 mL water).

2. **pH Adjustment:** Adjust the pH of the GO solution to 10 using a NaOH solution. This alkaline environment facilitates the reduction process.

3. **Chemical Addition:** Add 2 mL of hydrazine to the solution. Hydrazine acts as the reducing agent, removing oxygen functionalities from the GO.

4. **Sonication:** Subject the mixture to ultrasound irradiation for 2 hours. The sonication is conducted without cooling, allowing the temperature to reach around 60 ± 5°C. This process provides the energy required for the reduction reaction and exfoliation of graphene sheets.

5. **Purification:** Wash the obtained graphene nanosheets thoroughly with distilled water to remove residual chemicals and byproducts. Centrifuge the solution at 12,000 rpm for 10 minutes to separate the graphene from the water. Repeat this washing and centrifugation process several times to ensure the removal of impurities.

6. **Drying:** Dry the purified graphene nanosheets in a hot air oven.

**Result:** The process yields graphene or reduced graphene oxide nanosheets, referred to as US-rGO in the context.

**Note:** This method utilizes the chemical reduction of GO by hydrazine, assisted by the energy input from ultrasound. The sonochemical method is advantageous for its relatively fast reaction time and ability to produce well-dispersed graphene nanosheets.


'''],

    'contexts' : [['''Author:
Karthikeyan Krishnamoorthy

Doi:
https://doi.org/10.1016/j.ultsonch.2012.09.007

Method of producing graphene:
The as-synthesized GO nanosheets were used as the starting precursor for the synthesis of graphene via sonochemical method. Briefly, the pH of the as exfoliated GO solution (0.2 g in 200 mL) was adjusted to reach 10 by NaOH solution followed by the addition of 2 mL of hydrazine and the entire reaction is subjected to ultrasound irradiation for 2 h. The sonication reaction was conducted without any cooling, so that a temperature of about 60 ± 5oC was reached at the end of the reaction. The experiment is performed under atmospheric conditions. The obtained graphene nanosheets was washed thoroughly with distilled water and centrifuged at 12000 rpm for 10 min in order to remove the residuals. The procedure was repeated for several times until the synthesized product becomes free from trace amount of impurities. Further, the synthesized graphene nanosheets were dried in a hot air oven and were used for further characterization. The graphene or reduced graphene oxide nanosheet synthesized by ultrasonic method is named as US-rGO.
''',

'''Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and stirred for 48 h and separate filtrate and precipitate. Finally, precipitate was dried at 80oC for 24 h and characterized by using XRD and SEM-EDX, respectively.
''',

'''Author:
Mahbub Hassan

Doi:
https://doi.org/10.1016/j.jcis.2013.08.006

Method of producing graphene:
2.2. Synthesis of thermally expanded graphite (EG):
About 50 mg of graphite intercalated compound (GIC) was placed on a small ceramic boat and loaded into a moving-bed tubular furnace (OTF-1200X, MTI Corporation), preheated to 900-degree Celsius. The boat was placed at one end of the 0.4 m axial path of the furnace and moved gently during a 10 s period till it reached the oven exit. The thermally expanded exfoliated graphite was cooled to room temperature and removed from the boat for further processing.

2.3. Synthesis of graphene sheets:
The EG obtained is held by weak van der Waals forces. Thus, to obtain the desired graphene nanosheets (GNS), the EG was subjected to a final exfoliation stage. A dispersion was prepared in 20 mL distilled water by mixing 0.1 wt% EG with 0.3 wt% of sodium dodecyl sulfate powder. The resulting mixture was sonicated for 60min at room temperature. All sonication processes were carried out with a tip horn sonicator (Branson 450D) with a cylindrical tip (10 mm end cap diameter). The output power was fixed at 30 W to deliver energy at the 1650–1800 J/min level. The vial was placed in an ice bath during sonication to prevent any significant rise in temperature.


'''
]],

    'ground_truth' : ['The pH of the as exfoliated GO solution is adjusted to reach 10 by NaOH solution followed by the addition of 2 mL of hydrazine.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.2700, 'faithfulness': 0.6923}

## Test set from my FYP

###Question 1

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene using wheat straw, the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Feng Chen

Doi: 
https://doi.org/10.1016/j.jelechem.2016.02.035

Method of producing graphene:
Wheat straw was used as the starting material to obtain few-layer graphene by carbonization and graphitization of its hydrothermal product. Detailed steps are as follows: the raw material was first cut into small pieces (approximately 3 cm in length) and washed with distilled water, and then dried at 60 °C for 12 h before use. Within the subsequent hydrothermal treatment, 4 g of the washed wheat straw was immersed in a 400 mL Teflon-lined stainless steel autoclave wherein 280 mL of homogeneous 3 M KOH solution was cont

In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene using wheat straw, the steps should be in full proper sentences."],
    'answer': ['''To fabricate few-layer graphene from wheat straw, you can follow these detailed steps based on the hydrothermal method described by Feng Chen:

**Preparation:**

1. **Prepare the Wheat Straw:** Begin by cutting the raw wheat straw into small, manageable pieces, approximately 3 centimeters in length. Wash the cut straw thoroughly with distilled water to remove dirt and impurities. Afterward, dry the washed straw in an oven at 60°C for 12 hours.

**Hydrothermal Treatment:**

2. **Hydrothermal Reaction:**  Place 4 grams of the dried wheat straw into a Teflon-lined stainless steel autoclave. Add 280 mL of a 3 M potassium hydroxide (KOH) solution to the autoclave, ensuring it mixes well with the straw. Seal the autoclave tightly. Heat the autoclave to 150°C and maintain this temperature for 6 hours. This process breaks down the straw's structure.
3. **Cooling and Washing:** Allow the autoclave to cool down to room temperature naturally. After cooling, filter the resulting mixture using vacuum filtration with quantitative filter paper to collect the solid product. Wash the collected product thoroughly by ultrasonicating it in 120 mL of deionized water for 30 minutes. This removes excess KOH and byproducts. Dry the washed product in an oven at 80°C.

**Pyrolysis and Chemical Activation:**

4. **High-Temperature Treatment:** Transfer the dried product to a furnace capable of reaching high temperatures. Heat the product to 800°C at a rate of 5°C per minute under a constant flow of nitrogen gas (N2) at 150 cm3/min. Maintain the temperature at 800°C for 3 hours. This pyrolysis process converts the material into a carbonaceous structure.
5. **Acid Wash:** After cooling, wash the resulting carbonaceous product thoroughly with a 0.5 M hydrochloric acid (HCl) solution. This step removes residual KOH from the previous treatment. Dry the acid-washed product in a vacuum oven at 80°C for 12 hours to ensure complete drying.

**Graphitization:**

6. **Final Thermal Treatment:** Place the dried product into a graphite furnace. Heat the product rapidly to 2600°C in an argon (Ar) atmosphere and hold for 5 minutes. This high-temperature treatment induces graphitization, transforming the carbonaceous structure into few-layer graphene.

After these steps, you will obtain few-layer graphene derived from wheat straw. The typical yield for this method is approximately 11.3 wt.% of the initial wheat straw used.

'''],

    'contexts' : [['''Author:
Feng Chen

Doi:
https://doi.org/10.1016/j.jelechem.2016.02.035

Method of producing graphene:
Wheat straw was used as the starting material to obtain few-layer graphene by carbonization and graphitization of its hydrothermal product. Detailed steps are as follows: the raw material was first cut into small pieces (approximately 3 cm in length) and washed with distilled water, and then dried at 60 °C for 12 h before use. Within the subsequent hydrothermal treatment, 4 g of the washed wheat straw was immersed in a 400 mL Teflon-lined stainless steel autoclave wherein 280 mL of homogeneous 3 M KOH solution was contained, and then the autoclave was heated at a temperature of 150 °C for 6 h. When cooled down to room temperature naturally, the resulting samples were fetched out, collected by vacuum filtration with quantitative filter paper, washed by ultrasonication in 120 mL deionized water for 30 min and dried in an oven at 80 °C. The following high temperature pyrolysis and chemically activation was carried out at 800 °C for 3 h under the protection of N2 atmosphere at a heating rate of 5 °C min−1 and a N2 flow rate of 150 cm3 min−1 . After that, the carbonaceous product was thoroughly washed with 0.5 M HCl solution to remove the residual KOH and dried in a vacuum oven at 80 °C for 12 h. Finally, the asprepared product was thermally treated at 2600 °C for 5 min using a graphite furnace in an argon atmosphere. The typical yield of fewlayer graphene by using wheat straw as starting material was found to be ca. 11.3 wt.%.''',


'''Author:
F.S. Al-Hazmi

Doi:
https://doi.org/10.1016/j.synthmet.2014.12.028

Method of producing graphene:
5 g of graphite flakes (thickness ∼0.1 μm) was first immersed into 30 ml of ice cooled glutaric acid over night. The pretreated graphite flakes were filtered and left to dry at ambient conditions. The pretreated graphite subjected to a thermal shock at 700 °C for 1 min to expand the graphite. 500 mg of the expanded graphite flakes were dispersed in 50 ml methanol and sonicated for 10 min to ensure the complete dispersion of pretreated graphite in the methanol solution. The graphite solution was introduced into a Teflon autoclave with capacity 80 cm3 and put into microwave oven at power of 800 W and temperature 200 °C for 30 min. After this time, the reaction was terminated by put the Teflon autoclave into ice cooled bath. The supernatant was collected by centrifuging and washed several times with deionized water and HCl (10%). The graphene sheets were dried under vacuum over night at 70 °C.''',


'''Author:
Xuebin Wang

Doi:
10.1038/ncomms3905

Method of producing graphene:
Typically, 10 g glucose was mixed with 10 g ammonium salts (NH4Cl), which was then heated under a desired heating rate (4OC/min  was recommended) and finally treated at 1,350 OC for 3 h under Ar atmosphere in a tube furnace (50 cm length by 5 cm diameter). A black foam-like product, that is, (strutted graphene) SG, was collected. White granulated sugar (a kitchen-use sugar from Nissin-Sugar Co.) was also used to replace glucose to grow SG.




'''
]],

    'ground_truth' : ['Wheat straw was used as the starting material to obtain few-layer graphene by carbonization and graphitization of its hydrothermal product. Detailed steps are as follows: the raw material was first cut into small pieces (approximately 3 cm in length) and washed with distilled water, and then dried at 60 °C for 12 h before use. Within the subsequent hydrothermal treatment, 4 g of the washed wheat straw was immersed in a 400 mL Teflon-lined stainless steel autoclave wherein 280 mL of homogeneous 3 M KOH solution was contained, and then the autoclave was heated at a temperature of 150 °C for 6 h. When cooled down to room temperature naturally, the resulting samples were fetched out, collected by vacuum filtration with quantitative filter paper, washed by ultrasonication in 120 mL deionized water for 30 min and dried in an oven at 80 °C. The following high temperature pyrolysis and chemically activation was carried out at 800 °C for 3 h under the protection of N2 atmosphere at a heating rate of 5 °C/min and a N2 flow rate of 150 cm3/min . After that, the carbonaceous product was thoroughly washed with 0.5 M HCl solution to remove the residual KOH and dried in a vacuum oven at 80 °C for 12 h. Finally, the asprepared product was thermally treated at 2600 °C for 5 min using a graphite furnace in an argon atmosphere. The typical yield of fewlayer graphene by using wheat straw as starting material was found to be ca. 11.3 wt.%.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.8700, 'faithfulness': 0.7143}

### Question 2

In [ ]:
rag_query("Using the context given. Provide me a detailed step by step description on how to fabricate graphene, the number of layers would be 1 and the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Wei Wu

Doi: 
https://doi.org/10.1016/j.snb.2010.06.070

Method of producing graphene:
Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000oC and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. 

In [ ]:
data_samples = {
    'question': ["Using the context given. Provide me a detailed step by step description on how to fabricate graphene, the number of layers would be 1 and the steps should be in full proper sentences."],
    'answer': ['''To fabricate a single layer of graphene, the method outlined by Uk Sim et al. in their publication with DOI 10.1021/acsami.6b11750 provides a suitable approach:

1. **Prepare the Copper Substrate:** Begin with a clean copper foil, approximately 10 cm x 10 cm in size. Place this foil within a quartz tube.

2. **Anneal the Copper:** Heat the quartz tube containing the copper foil to 1000°C under a flow of hydrogen gas (H2) at a rate of 10 standard cubic centimeters per minute (sccm). Maintain this temperature for 60 minutes. This annealing process helps to increase the grain size of the copper, creating a more suitable substrate for graphene growth.

3. **Synthesize Graphene via Chemical Vapor Deposition (CVD):** Introduce a mixture of methane (CH4) and hydrogen (H2) gases into the quartz tube. Maintain a flow rate of 45 sccm for CH4 and 10 sccm for H2, keeping the pressure at 10 Torr. The methane acts as the carbon source for graphene growth. Hold these conditions for 30 minutes to allow for monolayer graphene formation on the copper surface.

4. **Cool Down and Remove Copper Foil:** Turn off the heater and allow the system to cool down. Once cooled, carefully remove the copper foil from the quartz tube.

5. **Protect the Graphene:** Immediately spin-coat a layer of poly(methyl methacrylate) (PMMA) onto the side of the copper foil where the graphene has grown. This PMMA layer serves as a protective layer during the subsequent transfer process.

6. **Remove Unwanted Graphene:** Using oxygen plasma etching (100 W RF power for 12 seconds), remove any graphene that may have deposited on the backside of the copper foil.

7. **Transfer the Graphene:** Carefully place the PMMA-coated copper foil into a solution of 0.1 M ammonium persulfate. This solution will etch away the copper, leaving the PMMA-protected graphene floating on the surface.

8. **Clean and Transfer to Substrate:** Thoroughly rinse the floating PMMA/graphene film in deionized water to remove any remaining etchant or copper residues. Carefully transfer the film onto a desired substrate, such as a p-type silicon wafer without a silicon oxide layer.

9. **Remove PMMA:** Finally, immerse the substrate with the transferred PMMA/graphene in acetone to dissolve the PMMA layer, leaving behind a pristine, single layer of graphene on your desired substrate.
'''],

    'contexts' : [['''Author:
Wei Wu

Doi:
https://doi.org/10.1016/j.snb.2010.06.070

Method of producing graphene:
Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000oC and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. After the Cu foil was completely etched away, graphene with PMMA/graphene film was transferred onto a Si wafer with 300 nm thermally grown SiO2. The PMMA was then removed by repeatedly rinsing the film in acetone.''',


'''Author:
F.S. Al-Hazmi

Doi:
https://doi.org/10.1016/j.synthmet.2014.12.028

Method of producing graphene:
5 g of graphite flakes (thickness ∼0.1 μm) was first immersed into 30 ml of ice cooled glutaric acid over night. The pretreated graphite flakes were filtered and left to dry at ambient conditions. The pretreated graphite subjected to a thermal shock at 700 °C for 1 min to expand the graphite. 500 mg of the expanded graphite flakes were dispersed in 50 ml methanol and sonicated for 10 min to ensure the complete dispersion of pretreated graphite in the methanol solution. The graphite solution was introduced into a Teflon autoclave with capacity 80 cm3 and put into microwave oven at power of 800 W and temperature 200 °C for 30 min. After this time, the reaction was terminated by put the Teflon autoclave into ice cooled bath. The supernatant was collected by centrifuging and washed several times with deionized water and HCl (10%). The graphene sheets were dried under vacuum over night at 70 °C.''',


'''Author:
Uk Sim

Doi:
10.1021/acsami.6b11750

Method of producing graphene:
Monolayer graphene was synthesized using thermal CVD. First, a copper (Cu) foil (10 × 10 cm2) was placed in a quartz tube and heated to 1000 °C for 60 min with flowing H2 at 10 sccm. Then, a gas mixture of 45 sccm CH4 and 10 sccm H2 flowed for 30 min under 10 Torr. After 30 min, the heater was turned off and removed from the Cu foil. Next, to handle the monolayer graphene on the front side of the Cu foil, a protective layer of poly(methyl methacrylate) (PMMA) was spin-coated on the graphene on the front side of the Cu foil, and the graphene on the back was removed using oxygen plasma (100 W RF power, 12 s). Then, the PMMA film on the graphene was removed using acetone, and the remaining Cu was etched away using a 0.1 M ammonium persulfate solution. Finally, the monolayer graphene was rinsed in deionized water and transferred onto a p-type silicon substrate without a silicon oxide layer. For the various stacked graphene layers, graphene with PMMA was floated on deionized water and transferred onto another graphene layer on Cu and etched/rinsed again. The transfer and etching/rinsing procedure was repeated for up to 4 layers of graphene. To synthesize graphene with defects, the monolayer graphene on the back side of the copper was removed using oxygen plasma, and the graphene on the front side of the Cu was treated with argon plasma (10 W RF power, 4 s). The PMMA layer was coated onto this graphene, and the Cu foil was removed. Then, the treated graphene was transferred onto silicon or used in the stacking process.




'''
]],

    'ground_truth' : ['Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000 ◦C and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. After the Cu foil was completely etched away, graphene with PMMA/graphene film was transferred onto a Si wafer with 300 nm thermally grown SiO2. The PMMA was then removed by repeatedly rinsing the film in acetone.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.8750, 'factual_correctness': 0.3100, 'faithfulness': 0.9000}

### Question 3

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene using rice husk, the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author:
Mohammad Shafri Ismail

Doi:
10.11113/mjfas.v15n4.1228

Method of producing graphene:
The synthesis process of rice husk-derived graphene was performed in accordance with the process described by Muramatsu et al. (2014). 5 g of raw rice husk was grounded and treatment inside Carbolite furnace for 2 hours at 400 °C. The solid residue obtained were called as rice husk ash (RHA). Then, the RHA was chemically activated using KOH with impregnation ratio of 1:2 of KOH powder. The mixture was then compacted in a ceramic crucible. Next, the crucible was placed in the midpoint of a larger ceramic crucible. The empty space

In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene using rice husk, the steps should be in full proper sentences."],
    'answer': ['''While I can't provide a detailed description of graphene fabrication solely from rice husk based on the provided contexts, the work by Mohammad Shafri Ismail offers a starting point for synthesizing graphene-like materials from this abundant resource. Here's a breakdown of their process, highlighting the need for further research and development:

**Step 1: Rice Husk Ash (RHA) Preparation:**
Begin by grinding approximately 5 grams of raw rice husk into a fine powder. This increases the surface area for the subsequent thermal decomposition process. Place the ground rice husk into a high-temperature furnace, like a Carbolite furnace, and heat it to 400°C for 2 hours. This controlled burning process removes organic matter, leaving behind a carbon-rich residue known as rice husk ash (RHA).

**Step 2: Chemical Activation:**
To enhance the porosity and reactivity of the RHA, chemical activation is employed.  Mix the RHA with potassium hydroxide (KOH) powder at a weight ratio of 1:2 (RHA:KOH). This mixture is then compacted into a ceramic crucible for the next stage.

**Step 3: High-Temperature Annealing:**
Place the crucible containing the RHA-KOH mixture inside a larger ceramic crucible. Fill the space between the two crucibles with sacrificial RHA, which acts as a barrier against oxidation during high-temperature treatment. Heat the entire setup in a furnace at 800°C for 2 hours. This annealing process initiates a series of chemical reactions, transforming the RHA into a graphene-like structure.

**Step 4: Purification and Washing:**
After annealing, carefully pour the contents of the inner crucible into 50 ml of deionized water. Stir the mixture thoroughly for 30 minutes to dissolve any remaining potassium ions. To separate the solid product, centrifuge the mixture at 3200 RPM for 30 minutes.

**Step 5: Sonication and Filtration:**
To further remove impurities and enhance the dispersion of the synthesized material, sonicate it in 100 ml of deionized water for 3 minutes at an amplitude of 50%. This process uses ultrasonic waves to break down agglomerates. Finally, filter the sonicated solution through a PTFE membrane with a pore size of 0.22 micrometers using vacuum filtration.

**Step 6: Drying:**
After filtration, transfer the collected material to a vacuum oven and dry it at 80°C for 24 hours. This removes residual moisture, yielding the final product.

**Important Considerations:**
It's crucial to understand that this process does not produce pristine, single-layer graphene like other methods. The resulting material is likely to be a mixture of few-layer graphene, graphene oxide, and potentially other carbon nanostructures. Characterization techniques like Raman spectroscopy, X-ray diffraction, and electron microscopy are essential to determine the exact composition and quality of the synthesized material.

Further research is needed to optimize this process, potentially exploring different activation agents, annealing temperatures, and purification steps, to achieve higher quality graphene from rice husk.


'''],

    'contexts' : [['''Author:
Mohammad Shafri Ismail

Doi:
10.11113/mjfas.v15n4.1228

Method of producing graphene:
The synthesis process of rice husk-derived graphene was performed in accordance with the process described by Muramatsu et al. (2014). 5 g of raw rice husk was grounded and treatment inside Carbolite furnace for 2 hours at 400 °C. The solid residue obtained were called as rice husk ash (RHA). Then, the RHA was chemically activated using KOH with impregnation ratio of 1:2 of KOH powder. The mixture was then compacted in a ceramic crucible. Next, the crucible was placed in the midpoint of a larger ceramic crucible. The empty space around ceramic crucible was fully covered by sufficiently pressing and packing the sacrificial RHA. The sacrificial RHA used to prevent oxidation on the mixture against air at high temperature. The crucibles were heated at 800°C for 2 hours. Treatment was carried out where the chemically activated RHA was poured in 50 ml deionized water (DI water) and stirred for 30 minutes to remove potassium ions residue. The sample was centrifuged at 3200 round per minutes (RPM) for 30 minutes. Then, the sample was sonicated in 100 ml of DI water for 3 minutes with amplitude of 50%. After that, the sample was rinsed in 50 ml DI water using vacuum filtration over a PTFE membrane with 0.22 mm pore size. Next, the sample was dried in vacuum oven for 24 hours at 80 °C to remove moisture.''',


'''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS''',


'''Author:
Hareema Saleem

Doi:
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was converted into mustard color. Washed the mixture with 1 M HCl and distilled water till the pH of supernatant liquid reached 7

2.2. Exfoliation of graphene oxide (GO):
To exfoliate the GO, sonicated the GO dispersion under normal condition for 30 min. The homogeneous brown dispersion was produced which was utilized for reduction. Dried the solution in the vacuum oven at 40 oC  for 24 h 2.3. Thermal reduction of graphene oxide The GO was annealed in a Nabertherm Gmbh N17/HR-400V Muffle heating furnace at 500 oC at a heating rate of 2 oC /min for 2 h and allowed to cool in furnace till the temperature decreased till 50 ± 5 oC. The GO was completely converted into reduced graphene oxide (rGO)





'''
]],

    'ground_truth' : ['The synthesis process of rice husk-derived graphene was performed in accordance with the process described by Muramatsu et al. (2014). As shown in Figure 4, 5 g of raw rice husk was grounded and treatment inside Carbolite furnace for 2 hours at 400 °C. The solid residue obtained were called as rice husk ash (RHA). Then, the RHA was chemically activated using KOH with impregnation ratio of 1:2 of KOH powder. The mixture was then compacted in a ceramic crucible. Next, the crucible was placed in the midpoint of a larger ceramic crucible. The empty space around ceramic crucible was fully covered by sufficiently pressing and packing the sacrificial RHA. The sacrificial RHA used to prevent oxidation on the mixture against air at high temperature. The crucibles were heated at 800°C for 2 hours. Treatment was carried out as shown in Figure 7 where the chemically activated RHA was poured in 50 ml deionized water (DI water) and stirred for 30 minutes to remove potassium ions residue. The sample was centrifuged at 3200 round per minutes (RPM) for 30 minutes. Then, the sample was sonicated in 100 ml of DI water for 3 minutes with amplitude of 50%. After that, the sample was rinsed in 50 ml DI water using vacuum filtration over a PTFE membrane with 0.22 mm pore size. Next, the sample was dried in vacuum oven for 24 hours at 80 °C to remove moisture.'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.5900, 'faithfulness': 0.5806}

### Question 4

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene using a SEKI AX5200S MPCVD system, the steps should be in full proper sentences. ")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
A. Kumar

Doi:
https://doi.org/10.1016/j.carbon.2011.11.033

Method of producing graphene:
Graphene film synthesis was performed in a SEKI AX5200S MPCVD system with H2 (50 sccm) and CH4 (5 sccm) as the feed gases. A 25 um thick copper foil (Alfa Aesar, 99.8% purity) was used as the substrate. The foil was cut in 2 · 2 cm2 pieces and placed into the MPCVD chamber, which was evacuated to a base pressure of 2 Torr and then filled with high-purity hydrogen at a pressure of 10 Torr. The Cu foil was supported by a ceramic stand that elevated it from the Mo puck by about 15 mm. The elevation of the Cu foil above Mo puc

In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene using a SEKI AX5200S MPCVD system, the steps should be in full proper sentences. "],
    'answer': ['''To fabricate graphene using a SEKI AX5200S MPCVD system, follow these steps:

1. **Prepare the Copper Substrate:** Procure a 25 μm thick copper foil with 99.8% purity. Cut the foil into 2 cm x 2 cm squares.

2. **Clean the Copper Substrate:** Place the copper square onto the ceramic stand within the MPCVD reaction chamber. Ensure the stand elevates the foil approximately 15 mm above the molybdenum puck to ensure optimal plasma coupling.

3. **Create a Hydrogen Atmosphere:** Seal the MPCVD chamber and evacuate it to a base pressure of 2 Torr. Introduce high-purity hydrogen gas into the chamber, maintaining a constant pressure of 10 Torr.

4. **Plasma Cleaning and Annealing:** Ignite the hydrogen plasma at 400 W power and maintain it for 3 minutes. This step will heat the copper foil to approximately 700°C and remove any oxide layer on its surface. The hydrogen flow rate should be kept at 50 sccm throughout this stage.

5. **Introduce Methane for Graphene Growth:** While maintaining the hydrogen plasma and its flow rate, introduce methane (CH4) into the chamber at a flow rate of 5 sccm. This will create a gas mixture with approximately 10% methane concentration, initiating graphene growth on the copper substrate. The growth duration can range from 30 seconds to 2 minutes depending on the desired graphene layer characteristics.

6. **Cool Down and Characterize:** After the desired growth time, turn off the plasma and allow the system to cool down to room temperature under a hydrogen atmosphere. Once cooled, carefully remove the graphene-coated copper substrate from the chamber. Characterize the synthesized graphene using techniques like Raman spectroscopy and atomic force microscopy to confirm its quality and layer number.

Note: This method, based on the work of A. Kumar (https://doi.org/10.1016/j.carbon.2011.11.033), highlights the key steps involved. The exact parameters such as growth duration and plasma power might need adjustments based on the specific system and desired graphene characteristics.



'''],

    'contexts' : [['''Author:
A. Kumar

Doi:
https://doi.org/10.1016/j.carbon.2011.11.033

Method of producing graphene:
Graphene film synthesis was performed in a SEKI AX5200S MPCVD system with H2 (50 sccm) and CH4 (5 sccm) as the feed gases. A 25 um thick copper foil (Alfa Aesar, 99.8% purity) was used as the substrate. The foil was cut in 2 · 2 cm2 pieces and placed into the MPCVD chamber, which was evacuated to a base pressure of 2 Torr and then filled with high-purity hydrogen at a pressure of 10 Torr. The Cu foil was supported by a ceramic stand that elevated it from the Mo puck by about 15 mm. The elevation of the Cu foil above Mo puck was found to be critical for the growth of the films because it ensures a strong coupling between the plasma and the Cu foil, thus enabling rapid self-heating of the foil by the microwave plasma. No additional heater was used. The thermocouple attached to the substrate table indicated a maximum reading of approximately 65oC during the process. A dual-wavelength infrared pyrometer indicated a maximum temperature of 700 ± 25 oC. However, as the copper foil is coated with graphene, the surface emissivity changes, making reliable IR measurements difficult. The entire growth is accomplished in less than 5 min and generally involves two stages. First, in a plasma cleaning and annealing step, the foil is kept in 400 W hydrogen plasma for 3 min. A H2 flow rate of 50 sccm was used with downstream pressure control to maintain 10 Torr chamber pressure throughout the process. This process leads to rapid self-heating and removal of copper oxide from the foil surface. The second stage involves introduction of CH4 (while maintaining the hydrogen plasma) at a flow of 5 sccm for a short duration (between 30 s and 2 min), providing approximately 10% CH4 concentration in hydrogen. Depending on the growth duration and whether or not annealing is used, different forms of carbon can be deposited''',


'''Author:
C Zhang

Doi:
10.1088/0957-4484/24/39/395603

Method of producing graphene:
Figure 1 illustrates the CVD reactor used in this work. The first high-temperature zone (the upstream one) was heated to 1050 ◦C and the second high-temperature zone (the downstream one) was heated to 1000 ◦C, while the temperature of the low-temperature zone (the middle region between the high-temperature zones) is ∼800 ◦C. In order to get a stable reaction atmosphere, a vacuum pump was used to remove the air inside the tube. As depicted in figure 1, two 8 × 12 cm2 strips of commercially available Cu foil of thickness 25 µm (99.8% purity) were respectively placed in the high-temperature zones and a SiO2 substrate (2 cm × 2 cm × 0.05 cm, with both sides polished) was spaced in the middle region of the low-temperature zone. Then the furnace was heated and pumped down to 1.0×10−6 Torr. The Cu foils surrounding the quartz tube were simultaneously annealed to 1000 ◦C in a 100 sccm H2 flow for 20 min to remove the remaining copper oxide. Subsequently, H2 (100 sccm) and CH4 (200 sccm) were supplied for 30 min growth. After growth, the furnace was cooled down to room temperature quickly by simply opening the furnace''',


'''Author:
Wei Wu

Doi:
https://doi.org/10.1016/j.snb.2010.06.070

Method of producing graphene:
Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000oC and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. After the Cu foil was completely etched away, graphene with PMMA/graphene film was transferred onto a Si wafer with 300 nm thermally grown SiO2. The PMMA was then removed by repeatedly rinsing the film in acetone.






'''
]],

    'ground_truth' : ['Graphene film synthesis was performed in a SEKI AX5200S MPCVD system with H2 (50 sccm) and CH4 (5 sccm) as the feed gases. A 25 um thick copper foil (Alfa Aesar, 99.8% purity) was used as the substrate. The foil was cut in 2 · 2 cm2 pieces and placed into the MPCVD chamber, which was evacuated to a base pressure of 2 Torr and then filled with high-purity hydrogen at a pressure of 10 Torr. The Cu foil was supported by a ceramic stand that elevated it from the Mo puck by about 15 mm (see Fig. 1(a)). The elevation of the Cu foil above Mo puck was found to be critical for the growth of the films because it ensures a strong coupling between the plasma and the Cu foil, thus enabling rapid self-heating of the foil by the microwave plasma. No additional heater was used. The thermocouple attached to the substrate table indicated a maximum reading of approximately 65oC during the process. A dual-wavelength infrared pyrometer indicated a maximum temperature of 700 ± 25oC. However, as the copper foil is coated with graphene, the surface emissivity changes, making reliable IR measurements difficult. The entire growth is accomplished in less than 5 min and generally involves two stages. First, in a plasma cleaning and annealing step, the foil is kept in 400 W hydrogen plasma for 3 min. A H2 flow rate of 50 sccm was used with downstream pressure control to maintain 10 Torr chamber pressure throughout the process. This process leads to rapid self-heating and removal of copper oxide from the foil surface. The second stage involves introduction of CH4 (while maintaining the hydrogen plasma) at a flow of 5 sccm for a short duration (between 30 s and 2 min), providing approximately 10% CH4 concentration in hydrogen. Depending on the growth duration and whether or not annealing is used, different forms of carbon can be deposited'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.9333, 'factual_correctness': 0.6200, 'faithfulness': 0.7742}

### Question 5

In [ ]:
rag_query("Using the context given. Provide me a detailed step by step description on how to fabricate graphene. I want to use waste PET bottles and the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Noha A. El Essawy

Doi: 
https://doi.org/10.1016/j.ecoenv.2017.07.014

Method of producing graphene:
First, waste PET bottles were crushed and sieved to obtain the desired size fractions (1–3 mm) using a conventional sieve-shaker. Then 2 g of raw PET waste was introduced into an enclosed 50 mL stainless steel autoclave reactor (SS316). The closed stainless steel reactor was placed inside the center of an electric furnace and heated to 800 °C with a rate of 8 °C/min and maintained at this temperature for 1 h. After that the system was left to cool overnight. The resulted dark products were collected and crushed



In [ ]:
data_samples = {
    'question': ["Using the context given. Provide me a detailed step by step description on how to fabricate graphene. I want to use waste PET bottles and the steps should be in full proper sentences."],
    'answer': ['''To fabricate graphene using waste PET bottles as a precursor, you can adapt the method described by El Essawy (https://doi.org/10.1016/j.ecoenv.2017.07.014):

1. **Prepare the PET waste:** Collect waste PET bottles and thoroughly clean them to remove any dirt or contaminants.
2. **Crush and sieve the PET:** Crush the cleaned PET bottles into smaller pieces. Use a sieve-shaker to obtain PET particles within a desired size range, ideally 1-3 mm. This size range has been found to be suitable for the subsequent steps.
3. **Weigh the PET sample:** Measure out a specific weight of the crushed PET particles, for example, 2 grams. This ensures consistency in your experiments.
4. **Load the reactor:** Carefully transfer the weighed PET particles into a stainless steel autoclave reactor. A 50 mL reactor made of SS316 stainless steel is suitable for this purpose.
5. **Heat the reactor:** Place the sealed reactor inside an electric furnace. Heat the furnace to a temperature of 800°C at a controlled rate of 8°C per minute. This gradual heating allows for better control over the process.
6. **Maintain the temperature:** Once the furnace reaches 800°C, maintain this temperature for a duration of 1 hour. This extended heating at high temperature facilitates the conversion of PET into graphene.
7. **Cool the system:** After the 1-hour holding time, turn off the furnace and allow the entire system to cool down naturally. Leaving it overnight is a recommended practice.
8. **Collect the product:** After the system has cooled down completely, carefully open the reactor. You will find a dark-colored product inside, which primarily consists of graphene.
9. **Further processing:** The collected product might require additional purification and characterization steps depending on the desired purity and application of the graphene.

It is important to note that this method provides a general outline. You might need to optimize parameters such as temperature, heating rate, and holding time based on the specific characteristics of your experimental setup and desired graphene properties.

'''],

    'contexts' : [['''Author:
Noha A. El Essawy

Doi:
https://doi.org/10.1016/j.ecoenv.2017.07.014

Method of producing graphene:
First, waste PET bottles were crushed and sieved to obtain the desired size fractions (1–3 mm) using a conventional sieve-shaker. Then 2 g of raw PET waste was introduced into an enclosed 50 mL stainless steel autoclave reactor (SS316). The closed stainless steel reactor was placed inside the center of an electric furnace and heated to 800 °C with a rate of 8 °C/min and maintained at this temperature for 1 h. After that the system was left to cool overnight. The resulted dark products were collected and crushed''',


'''Author:
Subash Sharma

Doi:
https://doi.org/10.1016/j.carbon.2014.01.051

Method of producing graphene:
A horizontal AP-CVD system with a quartz tube of length 90 cm and diameter of 50 mm was taken as the reactor placed in two split furnaces. Cu foil (Nilaco Corp.) of thickness 20 μm and purity 99.99% was used as substrate for graphene synthesis. Solid waste plastic used for material packaging is used as carbon source in these studies. Cu foil was cleaned by sonicating in acetone for 30 min and placed in the growth furnace without further treatment. Cu substrate was annealed for 30 min in 100 standard cubic centimeters per minute (sccm) of H2 at 1020 °C before the feedstock was introduced. Annealing is an important step to enlarge Cu grain size, reduce lattice mismatch and dislocations, which help to limit number of nucleation sites. 3 mg of waste plastic as carbon source was placed in a ceramic boat inside the lower temperature furnace. Graphene growth was carried out for 90 minutes using a gas mixture of Ar and H2 with the flow rate 98 and 2.5 sccm, respectively. Supply of precursor was continuously increased during the growth process to provide sufficient carbon atoms for uninterrupted growth of graphene crystal. The injection rate of polymeric components contained in the waste plastic was changed by the rate of evaporation. Thereafter, it was allowed to cool at a cooling rate of (~16OC/min). Individual graphene crystal was transferred to SiO2/Si substrate by coating a poly(methyl methacrylate) (PMMA) supporting layer. Etching of underneath Cu foil was achieved by using a 50 mg/ml concentrated Fe(NO3)3 solution. Subsequently, PMMA/graphene stack layer was transferred to SiO2/Si substrate and PMMA coating was dissolved in acetone. Finally, transferred graphene was treated with a diluted nitric acid solution to remove the residual Fe(NO3)3 and dried in air.''',


'''Author:
Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS



'''
]],

    'ground_truth' : ['First, waste PET bottles were crushed and sieved to obtain the desired size fractions (1–3 mm) using a conventional sieve-shaker. Then 2 g of raw PET waste was introduced into an enclosed 50 mL stainless steel autoclave reactor (SS316). The closed stainless steel reactor was placed inside the center of an electric furnace and heated to 800 °C with a rate of 8 °C/min and maintained at this temperature for 1 h. After that the system was left to cool overnight. The resulted dark products were collected and crushed. As it was illustrated in Fig. 1, the bottle wastes were used as a precursor for the synthesis of graphene'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.8333, 'factual_correctness': 0.7200, 'faithfulness': 0.4333}

### Question 6

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene, the no. of layers would be few layers graphene and the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Chellachamy Anbalagan Amarnath

Doi: 
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS


Author: 


In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene, the no. of layers would be few layers graphene and the steps should be in full proper sentences."],
    'answer': ['''A detailed step-by-step description for fabricating few-layer graphene using a modified Hummer's method and solvent exfoliation:

**Materials:**

* Graphite flakes
* Pyrrole
* Dimethylformamide (DMF)
* Ethanol
* Isopropanol
* Milli-Q water

**Procedure:**

1. **Prepare Graphite Oxide:** Utilize a modified Hummer's method to oxidize graphite flakes and obtain graphite oxide. This typically involves treating graphite with strong oxidizing agents in a controlled environment.
2. **Disperse Graphite Oxide:** Disperse 10.0 mg of the prepared graphite oxide flakes in 20 ml of Milli-Q water. Stir the mixture to ensure uniform dispersion, resulting in a brown graphite oxide solution.
3. **Introduce Pyrrole:** Add 2.0 ml of pyrrole to the graphite oxide dispersion. Pyrrole will act as an intercalating agent, aiding in the exfoliation of graphite oxide into graphene layers.
4. **Heat and Stir:** Stir the mixture vigorously at 95°C for 12 hours. The elevated temperature and stirring facilitate the intercalation of pyrrole between the graphite oxide layers.
5. **Filtration and Washing:** After the reaction, filter the resulting black mixture using a suitable filter paper. Wash the collected residue thoroughly with Milli-Q water to remove any unreacted pyrrole, residual graphite oxide, or other impurities.
6. **Drying:** After washing, obtain the black residue, which consists of few-layer graphene sheets. Dry the residue in a vacuum oven or under ambient conditions until it is completely dry.

This method provides a relatively simple approach to producing few-layer graphene by leveraging the intercalation and exfoliation properties of pyrrole. The resulting graphene sheets can be further characterized and processed for various applications.

**Note:** This process is adapted from the context provided by author Chellachamy Anbalagan Amarnath (https://doi.org/10.1016/j.carbon.2011.04.048).

'''],

    'contexts' : [['''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS
''',

'''Author:
Wei Wu

Doi:
https://doi.org/10.1016/j.snb.2010.06.070

Method of producing graphene:
Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000oC and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. After the Cu foil was completely etched away, graphene with PMMA/graphene film was transferred onto a Si wafer with 300 nm thermally grown SiO2. The PMMA was then removed by repeatedly rinsing the film in acetone.
''',

'''Author:
F.S. Al-Hazmi

Doi:
https://doi.org/10.1016/j.synthmet.2014.12.028

Method of producing graphene:
5 g of graphite flakes (thickness ∼0.1 μm) was first immersed into 30 ml of ice cooled glutaric acid over night. The pretreated graphite flakes were filtered and left to dry at ambient conditions. The pretreated graphite subjected to a thermal shock at 700 °C for 1 min to expand the graphite. 500 mg of the expanded graphite flakes were dispersed in 50 ml methanol and sonicated for 10 min to ensure the complete dispersion of pretreated graphite in the methanol solution. The graphite solution was introduced into a Teflon autoclave with capacity 80 cm3 and put into microwave oven at power of 800 W and temperature 200 °C for 30 min. After this time, the reaction was terminated by put the Teflon autoclave into ice cooled bath. The supernatant was collected by centrifuging and washed several times with deionized water and HCl (10%). The graphene sheets were dried under vacuum over night at 70 °C.



'''
]],

    'ground_truth' : ['Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS'],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.8571, 'factual_correctness': 0.5000, 'faithfulness': 0.3043}

### Question 7

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene, the no. of layers would be multi-layers graphene and the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Wei Wu

Doi: 
https://doi.org/10.1016/j.snb.2010.06.070

Method of producing graphene:
Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000oC and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. 

In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene, the no. of layers would be multi-layers graphene and the steps should be in full proper sentences."],
    'answer': ['''One method for fabricating multi-layered graphene utilizes a modified Hummers' method followed by a reaction with pyrrole.

**Step 1: Graphite Oxide Preparation:** Begin by preparing graphite oxide flakes using a modified Hummers' method. This method typically involves the oxidation of graphite using strong oxidizing agents.

**Step 2:  Dispersion of Graphite Oxide:** Disperse 10.0 mg of the prepared graphite oxide flakes in 20 ml of water. This should form a brown-colored graphite oxide dispersion.

**Step 3: Addition of Pyrrole:** Add 2.0 ml of pyrrole to the prepared graphite oxide dispersion. Pyrrole will act as a reducing agent and also help in the exfoliation of graphite oxide.

**Step 4: Reaction at Elevated Temperature:** Stir the mixture of graphite oxide dispersion and pyrrole for 12 hours at 95°C. This step facilitates the reduction of graphite oxide and the formation of multi-layered graphene.

**Step 5: Filtration and Washing:** After the reaction, filter the resulting black mixture. This separates the solid graphene material from the liquid. Wash the black residue thoroughly with water to remove any unreacted chemicals or byproducts.

**Step 6: Drying:** Dry the obtained black residue, which consists of multi-layered graphene sheets. This can be achieved by air drying or using a vacuum oven at a low temperature.

This method, reported by Chellachamy Anbalagan Amarnath et al., describes a chemical approach to exfoliate graphite oxide and obtain multi-layered graphene.
'''],

    'contexts' : [['''Author:
Wei Wu

Doi:
https://doi.org/10.1016/j.snb.2010.06.070

Method of producing graphene:
Briefly, the procedure is described as follows. Graphene was grown by thermal CVD on a Cu foil, at a temperature of 1000oC and under 1 atm pressure with methane as the precursor gas. A quartz tube with 2 diameter was used as the reaction chamber for our CVD system. Cu foils were rolled up in a roll, but without the rolled-up layers touching to each other. Following the growth, PMMA was spun on graphene/Cu substrate to form PMMA/graphene/Cu sandwich structure. Later, Cu foil was etched away using an iron nitrate aqua solution. After the Cu foil was completely etched away, graphene with PMMA/graphene film was transferred onto a Si wafer with 300 nm thermally grown SiO2. The PMMA was then removed by repeatedly rinsing the film in acetone.
''',

'''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS

''',

'''Author:
F.S. Al-Hazmi

Doi:
https://doi.org/10.1016/j.synthmet.2014.12.028

Method of producing graphene:
5 g of graphite flakes (thickness ∼0.1 μm) was first immersed into 30 ml of ice cooled glutaric acid over night. The pretreated graphite flakes were filtered and left to dry at ambient conditions. The pretreated graphite subjected to a thermal shock at 700 °C for 1 min to expand the graphite. 500 mg of the expanded graphite flakes were dispersed in 50 ml methanol and sonicated for 10 min to ensure the complete dispersion of pretreated graphite in the methanol solution. The graphite solution was introduced into a Teflon autoclave with capacity 80 cm3 and put into microwave oven at power of 800 W and temperature 200 °C for 30 min. After this time, the reaction was terminated by put the Teflon autoclave into ice cooled bath. The supernatant was collected by centrifuging and washed several times with deionized water and HCl (10%). The graphene sheets were dried under vacuum over night at 70 °C.



'''
]],

    'ground_truth' : ['''Graphite oxide was synthesized using graphite flakes by the Hummers method. GO was achieved by ultrasonication of graphite oxide in water for 2 h. In a typical experiment, 0.8 g Zn powder and 10 mL HCl (35 wt%) were added into 40 mL GO (1 mg/mL) solution. The mixture was stirred for 30 min. After that, 10 mL HCl (35 wt%) was added into the above solution and then maintained for a period of time to remove excess Zn powder. Finally, the resulting RGO was collected with filtration, washed with pure water and ethanol several times, and dried at 90oC for 12 h in a vacuum oven.'''],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.1667, 'factual_correctness': 0.0000, 'faithfulness': 0.4400}

### Question 8

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene using reduced graphene oxide, the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author: 
Hareema Saleem

Doi:  
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was c

In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene using reduced graphene oxide, the steps should be in full proper sentences."],
    'answer': ['''To fabricate graphene using reduced graphene oxide, you can follow these steps, adapted from the chemical exfoliation and thermal reduction method described by Hareema Saleem (https://doi.org/10.1016/j.matchemphys.2017.10.020):

**Step 1: Chemical Exfoliation of Graphite:**

1. **Prepare the oxidizing mixture:** In a suitable container, carefully combine 3 g of graphite flakes with a mixture of 360 ml concentrated H2SO4 and 40 ml concentrated H3PO4.
2. **Initiate oxidation:** Slowly add 18 g of KMnO4 to the acid mixture while stirring continuously. This reaction is exothermic, so ensure proper cooling to maintain a temperature between 35-40°C.
3. **Oxidize the graphite:** Maintain stirring at 35-40°C for 4-5 hours. Afterward, increase the temperature to 50°C and continue stirring for another 12 hours.
4. **Quench the reaction:** Carefully pour 400 ml of ice-cold distilled water into the mixture, followed by 5 ml of hydrogen peroxide (H2O2). This step stops the oxidation and changes the color of the mixture from dark pink to mustard.
5. **Wash and purify:** Wash the resulting mixture multiple times with 1 M HCl and then distilled water. Repeat this washing process until the supernatant liquid reaches a pH of 7, indicating removal of impurities.

**Step 2: Exfoliation of Graphene Oxide:**

1. **Sonicate the mixture:**  Disperse the obtained graphene oxide in distilled water and sonicate the mixture under standard conditions for approximately 30 minutes. This process breaks down larger particles and creates a homogeneous brown dispersion.
2. **Dry the dispersion:** Transfer the sonicated dispersion to a vacuum oven and dry at 40°C for 24 hours to obtain a graphene oxide film.

**Step 3: Thermal Reduction of Graphene Oxide:**

1. **Anneal the graphene oxide:** Place the dried graphene oxide film in a muffle furnace and heat it to 500°C at a rate of 2°C/minute. Maintain this temperature for 2 hours.
2. **Cool down:** After annealing, allow the furnace to naturally cool down to 50±5°C before removing the reduced graphene oxide (rGO) product.

This process effectively reduces the graphene oxide, removing oxygen-containing functional groups and restoring the structure closer to pristine graphene.

It is important to note that this is just one method for producing graphene from reduced graphene oxide. Other methods, such as chemical reduction using hydrazine or other reducing agents, can also be employed. The choice of method depends on the desired properties of the final graphene product and available resources.

'''],

    'contexts' : [['''Author:
Hareema Saleem

Doi:
https://doi.org/10.1016/j.matchemphys.2017.10.020

Method of producing graphene:
2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was converted into mustard color. Washed the mixture with 1 M HCl and distilled water till the pH of supernatant liquid reached 7

2.2. Exfoliation of graphene oxide (GO):
To exfoliate the GO, sonicated the GO dispersion under normal condition for 30 min. The homogeneous brown dispersion was produced which was utilized for reduction. Dried the solution in the vacuum oven at 40 oC  for 24 h 2.3. Thermal reduction of graphene oxide The GO was annealed in a Nabertherm Gmbh N17/HR-400V Muffle heating furnace at 500 oC at a heating rate of 2 oC /min for 2 h and allowed to cool in furnace till the temperature decreased till 50 ± 5 oC. The GO was completely converted into reduced graphene oxide (rGO)
''',

'''Author:
Kaveri Satheesh

Doi:
https://doi.org/10.1016/j.matlet.2013.09.044

Method of producing graphene:
Synthesis of GO: Graphite Oxide (GO) was synthesized from graphite powder by using Hummers method. 2 g of graphite powder was added to a mixture of 46 ml of concentrated H2SO4 and 1 g of NaNO3 in an ice bath. Then 6 g of KMnO4 was added slowly to the suspension under stirring to avoid any violent or explosive reactions and the temperature of the mixture was kept below 10oC by cooling. After adding the KMnO4 to the mixture, the dark green suspension was removed from the ice bath and the temperature was raised to 35 oC and maintained 1 h, which resulted in a thick paste. Then the mixture was diluted by 92 ml of water. Because of the addition of water in concentrated sulfuric acid medium a large amount of heat was released. Then the temperature was raised to 98 oC and maintained for 2 h. The reaction was terminated by the addition of 280 ml of water and 30% H2O2, resulting in brown mixture. Then the mixture was filtered and washed with 10% HCl, ethanol and water. The resulting solid was dried in a vacuum oven at 60 oC for 12 h. Then GO was obtained in the form of dry brown powder.

Synthesis of graphene sheets: Typically, 100 mg of GO was dispersed in 100 ml of water through ultrasonication for 1 h to yield GO dispersion. 0.5 mol of thiourea was added to the dispersion under stirring at 95 oC for 12 h. With the reduction process, the brown color of GO dispersion turns into black color. Finally the black color product (rGO) was washed with water and ethanol. The filtered product was dried in a vacuum oven at 60 oC for 12 h.
''',

'''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS




'''
]],

    'ground_truth' : ['''2.1. Chemical exfoliation method:
Oxidized the graphite flakes by chemical exfoliation method. Mixed 3 g of graphite flakes with 9:1 mixture of concentrated H2SO4/H3PO4 (360:40 ml) and 18 g of KMnO4. Stirred the mixture mechanically at 35-40oC for 4 to 5 h to produce the slightly exothermic reaction. Heated and stirred it at 50 oC for 12 h. Then cooled it to room temperature and poured 400 ml ice of distilled water and 5 ml hydrogen peroxide H2O2 in a mixture to complete the reaction. The dark pink color was converted into mustard color. Washed the mixture with 1 M HCl and distilled water till the pH of supernatant liquid reached 7

2.2. Exfoliation of graphene oxide (GO):
To exfoliate the GO, sonicated the GO dispersion under normal condition for 30 min. The homogeneous brown dispersion was produced which was utilized for reduction. Dried the solution in the vacuum oven at 40 oC  for 24 h 2.3. Thermal reduction of graphene oxide The GO was annealed in a Nabertherm Gmbh N17/HR-400V Muffle heating furnace at 500 oC at a heating rate of 2 oC /min for 2 h and allowed to cool in furnace till the temperature decreased till 50 ± 5 oC. The GO was completely converted into reduced graphene oxide (rGO)


'''],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'factual_correctness': 0.7000, 'faithfulness': 0.6857}

### Question 9

In [ ]:
rag_query("Give me a detailed step by step description on how to fabricate graphene using Silverson L5M rotor-stator mixer, the steps should be in full proper sentences.")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and s

In [ ]:
data_samples = {
    'question': ["Give me a detailed step by step description on how to fabricate graphene using Silverson L5M rotor-stator mixer, the steps should be in full proper sentences."],
    'answer': ['''While I can provide a general approach to producing graphene using a rotor-stator mixer like the Silverson L5M, I cannot give you a specific step-by-step recipe.  Here's why:

* **Lack of Information:** The question doesn't specify a starting material (graphite, graphite oxide, etc.) or desired type of graphene.  These factors heavily influence the procedure.
* **Complexity of the Process:**  Graphene production involves multiple chemical reactions, precise conditions (temperature, time), and purification steps. Simply using a mixer without the full context of the chosen method will not yield graphene.
* **Contextual Relevance:**  None of the provided contexts specifically use a Silverson L5M or a similar rotor-stator mixer for graphene production. While the mixer can be a helpful tool for mixing and dispersing materials in certain steps, it's not the primary method.

**However, here's a GENERAL outline of how a rotor-stator mixer MIGHT be incorporated into a graphene production workflow:**

1. **Preparation:** Choose a suitable graphene production method (e.g., modified Hummers method, electrochemical exfoliation). Gather all necessary chemicals and equipment.
2. **Mixing/Dispersion:**  The Silverson L5M could be used in steps requiring vigorous mixing and dispersion of materials.
    * Example:  If using graphite oxide, the mixer could help create a uniform dispersion in a solvent before further processing.
3. **Reaction:** The chosen method will dictate the specific reaction conditions.  The rotor-stator mixer may NOT be suitable for reactions involving strong acids, high temperatures, or other harsh environments.
4. **Purification:**  Graphene synthesis typically requires purification steps like washing, centrifugation, and filtration.  The mixer may not be directly involved in these steps.

**Important Considerations:**

* **Material Compatibility:** Ensure all materials used with the mixer are compatible with the chosen chemicals and solvents.
* **Safety Precautions:** Graphene production can involve hazardous materials.  Always wear appropriate PPE and follow established safety protocols.
* **Optimization:**  Graphene production is highly sensitive to process parameters. Extensive optimization and characterization will be needed to achieve the desired graphene quality.

**Recommendation:** I strongly advise consulting published research articles and protocols for specific graphene fabrication methods that align with your starting materials and desired graphene properties.


'''],

    'contexts' : [['''Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and stirred for 48 h and separate filtrate and precipitate. Finally, precipitate was dried at 80oC for 24 h and characterized by using XRD and SEM-EDX, respectively.
''',

'''Author:
F.S. Al-Hazmi

Doi:
https://doi.org/10.1016/j.synthmet.2014.12.028

Method of producing graphene:
5 g of graphite flakes (thickness ∼0.1 μm) was first immersed into 30 ml of ice cooled glutaric acid over night. The pretreated graphite flakes were filtered and left to dry at ambient conditions. The pretreated graphite subjected to a thermal shock at 700 °C for 1 min to expand the graphite. 500 mg of the expanded graphite flakes were dispersed in 50 ml methanol and sonicated for 10 min to ensure the complete dispersion of pretreated graphite in the methanol solution. The graphite solution was introduced into a Teflon autoclave with capacity 80 cm3 and put into microwave oven at power of 800 W and temperature 200 °C for 30 min. After this time, the reaction was terminated by put the Teflon autoclave into ice cooled bath. The supernatant was collected by centrifuging and washed several times with deionized water and HCl (10%). The graphene sheets were dried under vacuum over night at 70 °C.
''',

'''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS
'''
]],

    'ground_truth' : ['''Dispersions of graphene in aqueous surfactant solutions were prepared by shear mixing using a Silverson L5M rotor-stator mixer. A solution of sodium cholate (SC) in de-ionised water (1 g/L) was prepared and then added to natural flake at 100 g/L with a  volume of 1L. Graphene was exfoliated using a 32 mm rotor at 4500 rpm for 60 min. The resulting dispersion was left to settle overnight, and the supernatant discarded. Fresh surfactant solution was then added, and the exfoliation repeated as before. The dispersion was left to settle for several hours before subjecting the supernatant to the centrifugation-based size selection. To select nanosheets by size, we used controlled centrifugation with sequentially increasing rotation speeds as previously reported. An initial centrifugation at 1500rpm (240g, 2 h) was performed to remove unexfoliated material. The supernatant was subjected to further centrifugation at 2000rpm (425g, 2h). The sediment was collected in fresh water-SC (SC concentration 0.1 g/L to facilitate imaging), while the supernatant was subjected to further centrifugation at 2500rpm (665g, 2h). Again, the sediment was collected, and the supernatant centrifuged at higher rpm. This procedure was repeated for 3000rpm (958g, 2h), 5000rpm (2660g, 2h), 10000rpm (10 170g, 2h) and 18000rpm (32 600g, 2h) to yield samples with decreasing sizes in the respective sediments. Compared with a method based on taking the supernatant from a single step centrifugation, this produces dispersions with lower polydispersity. This greatly facilitates microscopic characterization required to accurately determine length and thickness. A Hettich Mikro 220R centrifuge equipped with a fixed angle rotor 1016 and sample aliquots of 10 mL was used for rotations up to 5000rpm. For the high-speed centrifugations, 1.5 mL vials and a rotor 1195-A was used. In the case of the analysis of the stock-like dispersion, the sample of Asbury graphite exfoliated in SC was centrifuged at 1 krpm (104g, 2 h) and the sediment discarded. We note that the removal of non-exfoliated graphite is required for the metrics to be applicable.
'''],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.0000, 'factual_correctness': 0.3100, 'faithfulness': 0.4828}

### Question 10

In [ ]:
rag_query("Give me a step by step instruction on how to produce graphene using the most common method")

> Input

INSTRUCTION:
Answer the QUESTION provided below. Multiple CONTEXTS are given and each context are formatted with an author, DOI and method of producing graphene.
Contexts given are to HELP guide your answer but by no means are they the expected or correct answer;
CHOOSE ONE context that is most relevant to the question to HELP supplement your answer.

CONTEXT:
Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and s

In [ ]:
data_samples = {
    'question': ["Give me a step by step instruction on how to produce graphene using the most common method"],
    'answer': ['''The most common method for producing graphene on a larger scale is the **modified Hummers' method**, followed by a reduction process. Here's a breakdown of the steps:

**Context:** While none of the provided contexts precisely detail the modified Hummers' method, the context by **Chellachamy Anbalagan Amarnath (DOI: https://doi.org/10.1016/j.carbon.2011.04.048)** mentions using a modified Hummers' method for preparing graphite oxide flakes. This method serves as the foundation for most graphene production.

**Step 1: Oxidation of Graphite (Modified Hummers' Method)**

1. **Prepare the oxidizing mixture:** In a well-ventilated area (due to hazardous fumes), carefully combine concentrated sulfuric acid (H2SO4) and phosphoric acid (H3PO4) in a ratio of 9:1. Cool the mixture in an ice bath.
2. **Add graphite:** Slowly add graphite powder to the acid mixture while maintaining a low temperature (below 10°C) to prevent overheating and potential hazards.
3. **Add potassium permanganate:** Gradually add potassium permanganate (KMnO4) to the mixture while stirring continuously. The mixture will turn green and then brown as the oxidation proceeds. This step should also be performed at a low temperature.
4. **Stir and react:** Allow the reaction to proceed for several hours, typically 12-24 hours, while maintaining a temperature around 35-50°C.
5. **Quench the reaction:** Carefully pour the mixture into a large volume of ice water containing a small amount of hydrogen peroxide (H2O2). This step helps to decompose any unreacted KMnO4 and stop the oxidation process.
6. **Washing and purification:** Wash the resulting graphite oxide extensively with deionized water, hydrochloric acid (HCl), and ethanol to remove residual acids and impurities.  This usually involves multiple centrifugation and washing cycles.

**Step 2: Reduction of Graphite Oxide to Graphene**

1. **Exfoliation:** The graphite oxide is dispersed in water and subjected to ultrasonication. This process exfoliates the layers of graphite oxide, forming a solution of graphene oxide (GO) sheets.
2. **Reduction:**  The GO needs to be reduced to remove the oxygen-containing functional groups and restore the electrical conductivity closer to that of pristine graphene. There are several methods for this, including:
    * **Chemical Reduction:** Using reducing agents like hydrazine, ascorbic acid, or hydroiodic acid.
    * **Thermal Reduction:** Heating GO at high temperatures (typically > 500 °C) in an inert atmosphere.
3. **Purification:**  The reduced graphene oxide (rGO) is further purified through washing and filtration to remove any remaining byproducts from the reduction process.

**Result:** The final product is a dispersion or powder of graphene, which can be further processed depending on the desired application.

**Important Notes:**

* **Safety First:**  The Hummers' method involves handling strong acids and oxidizers. Always wear appropriate personal protective equipment (PPE) and work in a well-ventilated area.
* **Scalability and Quality:** This method allows for the production of larger quantities of graphene. However, the quality of the graphene produced can vary depending on the specific reaction conditions and the reduction method used.
* **Environmental Considerations:**  The modified Hummers' method generates a significant amount of acidic waste. Proper disposal and exploration of greener alternatives are crucial for sustainable graphene production.


'''],

    'contexts' : [['''Author:
Rikson Siburian

Doi:
10.13005/ojc/340120

Method of producing graphene:
0.2 g of graphite powder was inserted into 250 mL erlenmeyer, then it was added 0.2 g NaNO3 and 15mL H2SO4 96%, respectively and stirred for 2 hours. After that, it was gradually added 1 g of KMnO4 and stirred for 24 h to generate graphite solution. Subsequently, the solution was added 20 mL of H2SO4 5% and 1 mL of 30% H2O2, respectively and stirred for one hour. Then, the solution was centrifuged at a speed of 3000 Rotation per Minute (RPM) for 20 min. to separate the filtrate and supernatant. The filtrate was added 10 mL ammonia 10 M and stirred for 48 h and separate filtrate and precipitate. Finally, precipitate was dried at 80oC for 24 h and characterized by using XRD and SEM-EDX, respectively.
''',

'''Author:
Chellachamy Anbalagan Amarnath

Doi:
https://doi.org/10.1016/j.carbon.2011.04.048

Method of producing graphene:
Graphite flakes, pyrrole, DMF, ethanol, isopropanol, and other solvents were used as received from Sigma–Aldrich (South Korea). Milli-Q water was used as reaction medium. Graphite oxide flakes were prepared by a modification of Hummer’s method. In brief, 10.0 mg graphite oxide flakes was dispersed in 20 ml water to form a brown GO dispersion. This was then stirred with 2.0 ml pyrrole for 12 h at 95OC. The resulting black mixture was filtered and washed with water. The black residue was GS
''',

'''Author:
Xuebin Wang

Doi:
10.1038/ncomms3905

Method of producing graphene:
Typically, 10 g glucose was mixed with 10 g ammonium salts (NH4Cl), which was then heated under a desired heating rate (4OC/min  was recommended) and finally treated at 1,350 OC for 3 h under Ar atmosphere in a tube furnace (50 cm length by 5 cm diameter). A black foam-like product, that is, (strutted graphene) SG, was collected. White granulated sugar (a kitchen-use sugar from Nissin-Sugar Co.) was also used to replace glucose to grow SG.


'''
]],

    'ground_truth' : ['''Synthesis of GO: Graphite Oxide (GO) was synthesized from graphite powder by using Hummers method. 2 g of graphite powder was added to a mixture of 46 ml of concentrated H2SO4 and 1 g of NaNO3 in an ice bath. Then 6 g of KMnO4 was added slowly to the suspension under stirring to avoid any violent or explosive reactions and the temperature of the mixture was kept below 10oC by cooling. After adding the KMnO4 to the mixture, the dark green suspension was removed from the ice bath and the temperature was raised to 35 oC and maintained 1 h, which resulted in a thick paste. Then the mixture was diluted by 92 ml of water. Because of the addition of water in concentrated sulfuric acid medium a large amount of heat was released. Then the temperature was raised to 98 oC and maintained for 2 h. The reaction was terminated by the addition of 280 ml of water and 30% H2O2, resulting in brown mixture. Then the mixture was filtered and washed with 10% HCl, ethanol and water. The resulting solid was dried in a vacuum oven at 60 oC for 12 h. Then GO was obtained in the form of dry brown powder.

Synthesis of graphene sheets: Typically, 100 mg of GO was dispersed in 100 ml of water through ultrasonication for 1 h to yield GO dispersion. 0.5 mol of thiourea was added to the dispersion under stirring at 95 oC for 12 h. With the reduction process, the brown color of GO dispersion turns into black color. Finally the black color product (rGO) was washed with water and ethanol. The filtered product was dried in a vacuum oven at 60 oC for 12 h.
'''],

}
dataset = Dataset.from_dict(data_samples)
from ragas.run_config import RunConfig


results = evaluate(dataset=dataset, metrics=metrics)

results

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'context_recall': 0.0625, 'factual_correctness': 0.3600, 'faithfulness': 0.1190}